In [1]:
import os
import re
import math
import logging
import random
import functools

import sqlite3
import json
import pandas as pd
import numpy as np
import itertools as tt

#from collections import Counter
import matplotlib.pyplot as plt

logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %I:%M:%S')
logging.getLogger().setLevel(logging.INFO)
logging.info("Logging set.")

2023-11-08 06:06:06 INFO: Logging set.


Using Python 3.9.12

In [2]:
def get_integer_input(filename):
    retval = None
    with open(filename) as IN:
        retval = [int(x) for x in IN.read().splitlines()]
    logging.info("Got {} integers from {}.".format(len(retval),filename))
    return retval

def get_string_input(filename):
    retval = None
    with open(filename) as IN:
        retval = IN.read().splitlines()
    logging.info("Got {} lines from {}.".format(len(retval),filename))
    return retval

def reverse_graph(A):
    ''' Return the transpose of the adjency matrix A
            A is a dict of dicts;  A[x][y] = value
            retval[x][y] = A[y][x]
    '''
    retval = dict()
    for row in A.keys():
        retval[row] = dict()
    for row, cols in A.items():
        for col in cols.keys():
            retval[col][row] = A[row][col]
    return retval

def l1_norm(vec):
    return sum([abs(v) for v in vec])

def get_grid_index(r, c, n, m):
    return r*m + c

def get_grid_neighborhood(ind, n, diagonal=True):
    ''' Return the indices that are adjacent to [r,c] in a grid of size n. '''
    retval = []
    size = n*n
    aboveRow = [ind-n-1, ind-n, ind-n+1]
    belowRow = [ind+n-1, ind+n, ind+n+1]
    if diagonal == False:
        aboveRow = [ind-n]
        belowRow = [ind+n]
        
    for newInd in aboveRow:
        if 0 <= newInd and newInd < size and ind//n == (newInd//n) + 1:
            retval.append(newInd)
    for newInd in [ind-1, ind+1]:
        if 0 <= newInd and newInd < size and ind//n == newInd//n:
            retval.append(newInd)
    for newInd in belowRow:
        if 0 <= newInd and newInd < size and ind//n == (newInd//n) - 1:
            retval.append(newInd)
    return retval


# Day 1: Sonar Sweep

## 1a. How many measurements are larger than the previous measurement.

## 1b. Consider sums of a three-measurement sliding window.  How many sums are larger than the previous sum?

In [3]:
filename = 'prob01input.txt'
depths = get_integer_input(filename)
count = sum([1 for i in range(len(depths) - 1) if depths[i] < depths[i+1]])
logging.info("1a.  Count = {}".format(count,))
count = sum([1 for i in range(len(depths) - 3) if depths[i] < depths[i+3]])
logging.info("1b.  Count = {}".format(count,))

2023-11-08 06:06:06 INFO: Got 2000 integers from prob01input.txt.
2023-11-08 06:06:06 INFO: 1a.  Count = 1692
2023-11-08 06:06:06 INFO: 1b.  Count = 1724


# Day 2: Dive!

## 2a.  What do you get if you multiply your final horizontal position by your final depth?

## 2b.  Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course.  What do you get if you multiply your final horizontal position by your final depth?

In [4]:
# Reading Input  (e.g. 'forward 5', 'up 10', 'down 7', etc.)
filename = 'prob02input.txt'
moves = get_string_input(filename)
line_re = re.compile('^(\S+) ([\d]+)$')
commands = []
for m in moves:
    m1 = line_re.match(m)
    if m1 != None:
        commands.append([m1.groups()[0], int(m1.groups()[1])])
    else:
        logging.error("Could not parse: {}".format(m,))

# Begin solution
x, y1, y2, aim = 0, 0, 0, 0
for direction, val in commands:
    if direction == 'forward':
        x += val
        y2 += aim * val
    elif direction == 'up':
        y1 -= val
        aim -= val
    elif direction == 'down':
        y1 += val
        aim += val

logging.info("2a. Horizontal, Depth = {}, {}.  Product = {}".format(x, y1, x*y1))
logging.info("2b. Horizontal, Depth = {}, {}.  Product = {}".format(x, y2, x*y2))

2023-11-08 06:06:06 INFO: Got 1000 lines from prob02input.txt.
2023-11-08 06:06:06 INFO: 2a. Horizontal, Depth = 2165, 933.  Product = 2019945
2023-11-08 06:06:06 INFO: 2b. Horizontal, Depth = 2165, 738712.  Product = 1599311480


# Day 3: Binary Diagnostic

## 3a. What is the power consumption of the submarine?

## 3b. What is the life support rating of the submarine?

In [5]:
filename = 'prob03input.txt'
binnums = get_string_input(filename)

n, k = len(binnums), len(binnums[0])
gamma, epsilon = '', ''
for bit_index in range(k):
    if sum([int(b[bit_index]) for b in binnums]) > n/2:
        gamma += '1'
        epsilon += '0'
    else:
        gamma += '0'
        epsilon += '1'

ga, ep = int(gamma, 2), int(epsilon, 2)
logging.info("3a. gamma rate: {}, epsilon rate: {}, power consumption: {}".format(ga, ep, ga*ep))

# Begin Part Two
prefBinnums = binnums[:]
unprefBinnums = binnums[:]
oxygen, co2 = None, None
for bit_index in range(k+1):
    prefN = len(prefBinnums)
    if prefN == 1 and oxygen == None:
        oxygen = int(prefBinnums[0],2)
    elif prefN == 1:
        pass
    elif sum([int(b[bit_index]) for b in prefBinnums]) >= prefN/2:
        prefBinnums = [b for b in prefBinnums if b[bit_index] == '1']
    else: 
        prefBinnums = [b for b in prefBinnums if b[bit_index] == '0']

    unprefN = len(unprefBinnums)
    if unprefN == 1 and co2 == None:
        co2 = int(unprefBinnums[0],2)
    elif unprefN == 1:
        pass
    elif sum([int(b[bit_index]) for b in unprefBinnums]) >= unprefN/2:
        unprefBinnums = [b for b in unprefBinnums if b[bit_index] == '0']
    else: 
        unprefBinnums = [b for b in unprefBinnums if b[bit_index] == '1']

logging.info("3b. oxygen rate: {}, co2 rate: {}, life support: {}".format(oxygen, co2, oxygen*co2))

2023-11-08 06:06:06 INFO: Got 1000 lines from prob03input.txt.
2023-11-08 06:06:06 INFO: 3a. gamma rate: 3516, epsilon rate: 579, power consumption: 2035764
2023-11-08 06:06:06 INFO: 3b. oxygen rate: 3311, co2 rate: 851, life support: 2817661


# Day 4:  Giant Squid

## 4a. What will your final score be for the first board that wins?

## 4b. What will your final score be for the last board that wins?

In [6]:
filename = 'prob04input.txt'
lines = get_string_input(filename)
board_re = re.compile('^\s*\d+\s+\d+\s+\d+\s+\d+\s+\d+\s*$')
space_re = re.compile('^\s*$')

# First line contains numbers to be drawn.
nums = [int(x) for x in lines[0].split(',')]

# Remaining lines contain boards. Getting boards from input.
boards = []
board = []
for l in lines[2:]:
    if board_re.match(l) != None:
        board.append([int(x) for x in l.split()])
    elif space_re.match(l) != None:
        if len(board) == 5:
            boards.append(board)
            board = []

if len(board) == 5:  # get last board
    boards.append(board)
    board = []

    
def is_winner(board, drawn):
    ''' Check if the board passed in is a winner (not diagnol)
         board is a 5x5 integer matrix (a bingo board)
         drawn is a list of numbers drawn.
        Return False if not a winner, or tupler ['row'/'col',  integer]
    '''
    retval = False
    for row in range(5):
        winner = True
        for col in range(5):
            if board[row][col] not in drawn:
                winner = False
                break
        if winner == True:
            break
    if winner == True:
        retval = ['row', row]
        
    if retval == False:
        for col in range(5):
            winner = True
            for row in range(5):
                if board[row][col] not in drawn:
                    winner = False
                    break
            if winner == True:
                break
        if winner == True:
            retval = ['col', col]
    
    return retval

def score_board(board, draw, drawn):
    ''' Return score of board.
         score = sum of drawn values on board * most recent draw
    '''
    retval = 0
    for row in range(5):
        for col in range(5):
            if board[row][col] not in drawn:
                retval += board[row][col]
    return retval * draw


# 4a. Find and score the first board to win.
drawn = set()
for draw in nums:
    drawn.add(draw)
    for b in range(len(boards)):
        result = is_winner(boards[b], drawn)
        if result != False:
            break
    if result != False:
        break

logging.info("Fisrt Board {}, Draw {}, Result {}".format(b, draw, result))
#logging.info("Board {}".format(boards[b],))
#logging.info("Drawn {}".format(drawn,))
logging.info("4a: Score = {}".format(score_board(boards[b], draw, drawn),))

# 4b. Find and score the last board to win.
numBoards = len(boards)
winners = set()
drawn = set()
loserScore = None
for draw in nums:
    drawn.add(draw)
    for b in range(numBoards):
        if b in winners:
            continue
        else:
            result = is_winner(boards[b], drawn)
            if result != False:
                winners.add(b)
            if len(winners) == numBoards:
                loserScore = score_board(boards[b], draw, drawn)
                break
    if loserScore != None:
        break

logging.info("Final Board {}, Draw {}, Result {}".format(b, draw, result))
#logging.info("Board {}".format(boards[b],))
#logging.info("Drawn {}".format(drawn,))
logging.info("4b: Score = {}".format(loserScore,))


2023-11-08 06:06:06 INFO: Got 601 lines from prob04input.txt.
2023-11-08 06:06:06 INFO: Fisrt Board 93, Draw 39, Result ['row', 3]
2023-11-08 06:06:06 INFO: 4a: Score = 27027
2023-11-08 06:06:06 INFO: Final Board 21, Draw 87, Result ['row', 0]
2023-11-08 06:06:06 INFO: 4b: Score = 36975


# Day 5:  Hydrothermal Venture

## 5a. At how many points do at least two lines overlap (no diagnols)?

## 5b. At how many points do at least two lines overlap (with diagnols)?

In [7]:
filename = 'prob05input.txt'
lines = get_string_input(filename)

line_re = re.compile('^([\d]+),([\d]+) -> ([\d]+),([\d]+)$')
endpoints = []
for l in lines:
    m1 = line_re.match(l)
    if m1 == None:
        logging.error("Could not process line. {}".format(l,))
    else:
        endpoints.append([int(m1.groups()[0]),int(m1.groups()[1]),int(m1.groups()[2]),int(m1.groups()[3])])

m = max([max(p) for p in endpoints])

covered1 = []
covered2 = []

for i in range(m+1):
    covered1.append([0] * (m+1))
    covered2.append([0] * (m+1))
    
for x1, y1, x2, y2 in endpoints:
    if x1 == x2:
        miny, maxy = min([y1,y2]), max([y1,y2])
        for j in range(miny, maxy+1):
            covered1[x1][j] += 1
            covered2[x1][j] += 1
    elif y1 == y2:
        minx, maxx = min([x1,x2]), max([x1,x2])
        for i in range(minx, maxx+1):
            covered1[i][y1] += 1
            covered2[i][y1] += 1
    else:
        xadd = 1 if x1 < x2 else -1
        yadd = 1 if y1 < y2 else -1
        for i in range(abs(x1-x2) + 1):
            covered2[x1+(xadd*i)][y1+(yadd*i)] += 1

large1 = 0
large2 = 0
for i in range(m+1):
    for j in range(m+1):
        if covered1[i][j] > 1:
            large1 += 1
        if covered2[i][j] > 1:
            large2 += 1

logging.info("5a. Overlap = {}".format(large1,))
logging.info("5b. Overlap = {}".format(large2,))

2023-11-08 06:06:06 INFO: Got 500 lines from prob05input.txt.
2023-11-08 06:06:06 INFO: 5a. Overlap = 6113
2023-11-08 06:06:06 INFO: 5b. Overlap = 20373


# Day 6:  Lantern Fish

## 6a.  How many lanternfish would there be after 80 days?

## 6b.  How many lanternfish would there be after 256 days?

In [8]:
filename = 'prob06input.txt'
line = get_string_input(filename)
nums = [int(x) for x in line[0].split(',')]

def get_pop(fish):
    retval = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0}
    for x in fish:
        retval[x] += 1
    return retval

def breed2(fish):
    retval = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0}
    for x in fish.keys():
        if x == 0:
            retval[8] = fish[0]
            retval[6] += fish[0]
        else:
            retval[x-1] += fish[x]
    return retval

fish = get_pop(nums)
counts = []
for day in range(256):
    fish = breed2(fish)
    counts.append(sum([fish[x] for x in fish.keys()]))
    if day in [79, 255]:
        logging.info("After {} day: {}".format(day+1, counts[-1]))



2023-11-08 06:06:06 INFO: Got 1 lines from prob06input.txt.
2023-11-08 06:06:06 INFO: After 80 day: 396210
2023-11-08 06:06:06 INFO: After 256 day: 1770823541496


# Day 7:  The Treachery of Whales

## 7a.  How much fuel must they spend to align to that position?

## 7b.  How much fuel must they spend to align to that position?

In [9]:
filename = 'prob07input.txt'
line = get_string_input(filename)
starts = [int(x) for x in line[0].split(',')]

def triangle(x):
    return x*(x+1)/2

S = sum(starts)
n = len(starts)
F = [S]  # F[i] fuel to move all to i
for end in range(1, max(starts)+1):
    # F[i] = (cost to move to i-1) - (cost to move all right 1) + (2 * num of those starting left of end)
    F.append(F[end-1] - n + (2 * len([xi for xi in starts if xi < end])))
    if F[end] - F[end-1] >= 0:
        logging.info("7a. Minimum Value is {}".format(F[end-1],))
        break

F = [sum([triangle(s) for s in starts])]  # F[i] fuel to move all to i
for end in range(1, max(starts) + 1):
    # F[i] = (cost to move to i-1) - (magic) + (num of those starting left of end)
    F.append(F[end-1] - S + (n*(end-1)) + len([xi for xi in nums if xi < end]))
    if F[end] - F[end-1] >= 0:
        logging.info("7b. Minimum value is {}".format(round(F[end-1]),))
        break



2023-11-08 06:06:06 INFO: Got 1 lines from prob07input.txt.
2023-11-08 06:06:06 INFO: 7a. Minimum Value is 344138
2023-11-08 06:06:06 INFO: 7b. Minimum value is 94827739


# Day 8:  Seven Segment Search

## 8a.  In the output values, how many times do digits 1, 4, 7, or 8 appear?

## 8b.  What do you get if you add up all of the output values?

In [10]:
filename = 'prob08input.txt'
lines = get_string_input(filename)

def get_digits_dict(l):
    ''' 1 has length 2
        7 has length 3
        4 has length 4
        8 has length 7
        9 has length 6 and contains 1 and 4
        0 has length 6 and contains 1 but not 4
        6 has length 6 and does not contain 1
        5 has length 5 and is contained by 6
        3 has length 5 and contains 1
        2 is left over
    '''
    retval = {0:None,1:None,2:None,3:None,4:None,5:None,6:None,7:None,8:None,9:None}
    digit_str, output_str = l.split(" | ")
    # Set the values that correspond to digits 1, 7, 4, and 8
    for val in digit_str.split(" "):
        if len(val) == 2:       retval[1] = set(list(val))
        elif len(val) == 3:     retval[7] = set(list(val))
        elif len(val) == 4:     retval[4] = set(list(val))
        elif len(val) == 7:     retval[8] = set(list(val))
            
    # Set the values that correspond to digits 9, 6, and 0
    for val in digit_str.split(" "):
        if len(val) == 6:
            valset = set(list(val))
            if retval[4].issubset(valset):    retval[9] = valset
            elif retval[1].issubset(valset):  retval[0] = valset
            else:                             retval[6] = valset

    # Set the values that correspond to digits 9, 6, and 0
    for val in digit_str.split(" "):
        if len(val) == 5:
            valset = set(list(val))
            if retval[1].issubset(valset):    retval[3] = valset
            elif valset.issubset(retval[6]):  retval[5] = valset
            else:                             retval[2] = valset

    return retval

def get_digit(digits, s):
    ''' Return the digit such that digits[digit] = s (as sets)'''
    retval = None
    for d in digits:
        if len(digits[d]) == len(s):
            if digits[d].issubset(s):
                retval = d
                break
    return d
    
solA = 0
solB = 0
for l in lines:
    digits = get_digits_dict(l)
    output_str = l.split(" | ")[1]
    output_val = 0
    for i, out_dig in enumerate(output_str.split(" ")):
        output_val += ( (10**(3-i)) * get_digit(digits, set(list(out_dig))) )             
        if get_digit(digits, set(list(out_dig))) in [1,7,4,8]:
            solA += 1
    solB += output_val
logging.info("8a. The number of output digits that are in [1,4,7,8]: {}".format(solA,))
logging.info("8b. The sum is: {}".format(solB,))

2023-11-08 06:06:06 INFO: Got 200 lines from prob08input.txt.
2023-11-08 06:06:06 INFO: 8a. The number of output digits that are in [1,4,7,8]: 383
2023-11-08 06:06:06 INFO: 8b. The sum is: 998900


# Day 9:  Smoke Basin

## 9a.  What is the sum of the risk levels of all low points on your heightmap?

## 9b.  What do you get if you multiply together the sizes of the three largest basins?

In [11]:
filename = 'prob09input.txt'
lines = get_string_input(filename)
    
def is_low_point(A, p):
    ''' Return True if all neighbors are higher; otherwise False'''
    retval = True
    for i in get_grid_neighborhood(p, n, diagonal=False):
        if A[i] <= A[p]:
            retval = False
            break
    return retval

def get_basin(A, p):
    ''' Return (recursively) neighbors that have higher values but less than 9.'''
    retval = [p]
    for i in get_grid_neighborhood(p, n, diagonal=False):
        if A[p] < A[i] < 9:
            for e in get_basin(A, i):
                if e not in retval:
                    retval.append(e)
    return retval

A, n, m = dict(), len(lines), len(lines[0])
for r, c in tt.product(range(n), range(m)):
    A[get_grid_index(r, c, n, m)] = int(lines[r][c])

risk, basins = 0, []
for p in range(len(A)):
    if is_low_point(A, p):
        risk += (A[p] + 1)
        basins.append(len(get_basin(A, p)))

logging.info("9a. Sum of all risk of low points = {}".format(risk,))
basins = sorted(basins, reverse=True)
solution = basins[0] * basins[1] * basins[2]
logging.info("9b. Basin Solution = {}".format(solution,))

2023-11-08 06:06:06 INFO: Got 100 lines from prob09input.txt.
2023-11-08 06:06:06 INFO: 9a. Sum of all risk of low points = 537
2023-11-08 06:06:06 INFO: 9b. Basin Solution = 1142757


# Day 10:  Syntax Scoring

## 10a.  What is the total syntax error score for those errors?

## 10b.  What is the middle score?

In [12]:
filename = 'prob10input.txt'
lines = get_string_input(filename)

def is_corrupted(line):
    ''' If corrupted, return [False, offending char].  Else [True, leave]
         where leave is the set of unpaired chars.
    '''
    pairs = {'{':'}','[':']','(':')','<':'>'}
    retval = [False, None]
    poplist = []
    for i in range(len(line)):
        ch = line[i]
        if ch in ['{','[','(','<']:
            poplist.append(ch)
        elif pairs[ poplist[-1] ] == ch:
            poplist.pop()
        else:
            retval = [True, ch]
            break
            
    if retval[0] == False:
        retval[1] = poplist
        
    return retval

scoreA_dict = {')':3, ']':57, '}':1197, '>':25137}
scoreA = 0
scoreB_dict = {'(':1, '[':2, '{':3, '<':4}
scoresB = []
for line in lines:
    corrupted, leave = is_corrupted(line)
    if leave in [')',']','}','>']:
        scoreA += scoreA_dict[leave]
    else:
        score = 0
        while len(leave) > 0:
            score *= 5
            ch = leave.pop()
            score += scoreB_dict[ch]
        scoresB.append(score)
        
logging.info("10a. Total syntax error score = {}".format(scoreA,))
logging.info("10b. Winner (median) = {}".format(int(np.median(scoresB)),))

2023-11-08 06:06:07 INFO: Got 106 lines from prob10input.txt.
2023-11-08 06:06:07 INFO: 10a. Total syntax error score = 367227
2023-11-08 06:06:07 INFO: 10b. Winner (median) = 3583341858


# Day 11:  Dumbo Octopus

## 11a.  How many total flashes are there after 100 steps?

## 11b.  What is the first step during which all octopuses flash?

In [13]:
lines = ['3113284886',
'2851876144',
'2774664484',
'6715112578',
'7146272153',
'6256656367',
'3148666245',
'3857446528',
'7322422833',
'8152175168']

def step(A, n):
    ''' Increment all values in dict A.  Flash values if needed. 
        Return the number of indices that flashed.
    '''
    flashed = set()
    toFlash = []
    
    for i in range(n*n):
        A[i] += 1
        if A[i] > 9:
            flashed.add(i)
            toFlash.append(i)

    while len(toFlash) > 0:
        flasher = toFlash.pop(0)
        for j in get_grid_neighborhood(flasher, n, diagonal=True):
            if j not in flashed:
                A[j] += 1
                if A[j] > 9:
                    flashed.add(j)
                    toFlash.append(j)
        A[flasher] = 0
    return len(flashed)

A, n, m = dict(), len(lines), len(lines[0])
for r, c in tt.product(range(n), range(n)):
    A[get_grid_index(r, c, n, m)] = int(lines[r][c])

numFlashed = 0
for i in range(1000):
    stepFlashes = step(A, n)
    numFlashed += stepFlashes
    if i == 99:
        logging.info("11a. After 100 steps there are {} flashes.".format(numFlashed,))
    if stepFlashes == 100:
        logging.info("11b. All values flashed during step {}".format(i+1,))
        break


2023-11-08 06:06:07 INFO: 11a. After 100 steps there are 1705 flashes.
2023-11-08 06:06:07 INFO: 11b. All values flashed during step 265


# Day 12:  Passage Pathing

## 12a.  How many paths through this cave system are there that visit small caves at most once?

## 12b.  Given these new rules, how many paths through this cave system are there?

In [14]:
filename = 'prob12input.txt'
lines = get_string_input(filename)

E = dict()  # E[v] = list of neighbors of v
for l in lines:
    v1, v2 = l.split("-")
    N1 = E.get(v1,[])
    N2 = E.get(v2,[])
    if v1 not in N2:
        N2.append(v1)
        E[v2] = N2
    if v2 not in N1:
        N1.append(v2)
        E[v1] = N1

walks = [['start']]
finished = 0
while len(walks) > 0:
    w = walks.pop(0)      # Extended first (shortest) walk in queue.
    for v in E[w[-1]]:    # Get neighbors of final vertex
        if v == 'end':
            finished += 1
        elif 'A' <= v[0] <= 'Z' or \
             ('a' <= v[0] <= 'z' and v not in w):
            new_w = w.copy()
            new_w.append(v)
            walks.append(new_w)  # Add extended walk to end of queue

logging.info('12a. Number of "paths": {}'.format(finished,))

walks = [['start']]
finished = 0
while len(walks) > 0:
    w = walks.pop(0)
    for v in E[w[-1]]:
        if v == 'end':
            finished += 1
        elif 'A' <= v[0] <= 'Z':
            new_w = w.copy()
            new_w.append(v)
            walks.append(new_w)
        elif 'a' <= v[0] <= 'z' and v != 'start':
            # Add if first visit to v or if first lower case for second visit
            if v not in w or\
               max([w.count(x) for x in w if 'a' <= x[0] <= 'z']) < 2:
                new_w = w.copy()
                new_w.append(v)
                walks.append(new_w)

logging.info('12b. Number of "paths": {}'.format(finished,))

2023-11-08 06:06:07 INFO: Got 22 lines from prob12input.txt.
2023-11-08 06:06:07 INFO: 12a. Number of "paths": 3576
2023-11-08 06:06:11 INFO: 12b. Number of "paths": 84271


# Day 13:  Transparent Origami

## 13a.  How many dots are visible after completing just the first fold?

## 13b.  What code do you use to activate the infrared thermal imaging camera system?

In [15]:
filename = 'prob13input.txt'
lines = get_string_input(filename)
fold_re = re.compile("^fold along (x|y)=(\d+)\s*$")
garbage_re = re.compile("^\s*$")

# Read input.
dots = []
folds = []
for l in lines:
    m1 = fold_re.match(l)
    if len(l.split(",")) == 2:
        dots.append([int(x) for x in l.split(",")])
    elif m1 != None:
        folds.append([m1.groups(0)[0],int(m1.groups(0)[1])])
    elif garbage_re.match(l) == None:
        logging.error("Line = '{}'".format(l,))


def fold_paper(fold, dots):
    ''' Return a list of dots, each dot is [x,y] coordinate.
         fold = (axis, value) where axis is in ['x','y'] and value is an int; the line of the fold.
         dots = [[x1,y1], [x2,y2], ...]
    '''
    retval = []
    for d in dots:
        new_d = None
        if f[0] == 'x':
            if d[0] == f[1]:    continue
            elif d[0] < f[1]:   new_d = [d[0], d[1]]
            elif d[0] > f[1]:   new_d = [2*f[1]-d[0], d[1]]
        elif f[0] == 'y':
            if d[1] == f[1]:    continue
            elif d[1] < f[1]:   new_d = [d[0], d[1]]
            elif d[1] > f[1]:   new_d = [d[0], 2*f[1]-d[1]]

        if new_d is not None and new_d not in retval:
            retval.append(new_d)
    return retval

# Apply folds
for i, f in enumerate(folds):
    dots = fold_paper(f, dots)
    if i == 0:
        logging.info("13a. After the first fold there are {} dots.".format(len(dots),))

# Output the dots ()
xmax, ymax = max([d[0] for d in dots]), max([d[1] for d in dots])
M = ['.' * (xmax+1) for x in range(ymax+1)]
for d in dots:
    M[d[1]] = M[d[1]][:d[0]] + '#' + M[d[1]][d[0]+1:]
logging.info("13b. After all folds:\n{}".format('\n'.join(M),))
    

2023-11-08 06:06:11 INFO: Got 890 lines from prob13input.txt.
2023-11-08 06:06:11 INFO: 13a. After the first fold there are 729 dots.
2023-11-08 06:06:11 INFO: 13b. After all folds:
###...##..####.#....###..#..#.####.###.
#..#.#..#....#.#....#..#.#..#.#....#..#
#..#.#......#..#....###..####.###..#..#
###..#.##..#...#....#..#.#..#.#....###.
#.#..#..#.#....#....#..#.#..#.#....#...
#..#..###.####.####.###..#..#.#....#...


# Day 14:  Extended Polymerization

## 14a.  After 10 steps, what do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

## 14b.  After 40 steps, what do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [16]:
filename = 'prob14input.txt'
lines = get_string_input(filename)
polymer = lines[0]

map_re = re.compile("^([A-Z]{2}) -> ([A-Z])\s*$")
mapper = dict()
for l in lines[2:]:
    m1 = map_re.match(l)
    if m1 != None:
        mapper[m1.groups(0)[0]] = m1.groups(0)[1]

def apply_mapper(pairs, mapper):
    ''' Return a dict; retval[pair] = count or pair after mapper applied'''
    retval = dict()
    for p, c in pairs.items():
        p1, p2 = p[0] + mapper[p], mapper[p] + p[1]
        retval[p1] = retval.get(p1,0) + c
        retval[p2] = retval.get(p2,0) + c
    return retval


def count_letters(pairs, lastChar):
    ''' Return dict where retval[ch] = number of times ch appears in string.'''
    retval = dict()
    for p, c in pairs.items():
        p0 = p[0]
        retval[p0] = retval.get(p0,0) + c
    retval[lastChar] = retval.get(lastChar,0) + 1
    return retval


pairs = dict()
for i in range(len(polymer)-1):
    pairs[polymer[i:i+2]] = pairs.get(polymer[i:i+2],0) + 1
lastChar = polymer[-1]

for i in range(40):
    pairs = apply_mapper(pairs, mapper)
    if i + 1 == 10:
        counts = count_letters(pairs, lastChar)
        logging.info("14a. Solution {}".format(max(counts.values()) - min(counts.values()), ))
    if i + 1 == 40:
        counts = count_letters(pairs, lastChar)
        logging.info("14b. Solution {}".format(max(counts.values()) - min(counts.values()), ))


2023-11-08 06:06:11 INFO: Got 102 lines from prob14input.txt.
2023-11-08 06:06:11 INFO: 14a. Solution 2068
2023-11-08 06:06:11 INFO: 14b. Solution 2158894777814


# Day 15:  Chiton

## 15a.  What is the lowest total risk of any path from the top left to the bottom right?

## 15b.  Using the full map, what is the lowest total risk of any path from the top left to the bottom right?

In [17]:
filename = 'prob15input.txt'
lines = get_string_input(filename)
    
def find_lowest_risk_path(A, n, m):
    retval = None
    target = n * m - 1
    visited = set()
    distances = {0:0}
    possibles = [0]    # [index]
    while len(possibles) > 0:
        cur = possibles.pop(0)
        dist_to_cur = distances[cur]
        visited.add(cur)
        if cur == target:  # break condition
            retval = dist_to_cur
            break

        for ind in get_grid_neighborhood(cur, n, diagonal=False):
            if ind not in visited:
                dist_thru_cur = dist_to_cur + A[ind]
                if distances.get(ind, None) == None:
                    distances[ind] = dist_thru_cur    #   set tentative distance
                    possibles.append(ind)
                elif dist_thru_cur < distances[ind]:  # if shorter route through cur
                    distances[ind] = dist_thru_cur    #   reset tentative distance

        possibles = sorted(possibles, key=lambda x: distances[x])

    return retval

# Part a.
A1, n1, m1 = dict(), len(lines), len(lines[0])
for r, c in tt.product(range(n1), range(m1)):
    A1[get_grid_index(r, c, n1, m1)] = int(lines[r][c])
risk = find_lowest_risk_path(A1, n1, m1)
logging.info("15a. Lowest risk path: {}".format(risk,))

# Part b.
A2, n2, m2 = dict(), 5*n1, 5*m1
for r, c in tt.product(range(n2), range(m2)):
    ind = get_grid_index(r, c, n2, m2)
    A2[ind] = int(lines[r % n1][c % m1]) + r//n1 + c//m1
    while A2[ind] >= 10:
        A2[ind] -= 9
risk2 = find_lowest_risk_path(A2, n2, m2)
logging.info("15b. Lowest risk path: {}".format(risk2,))


2023-11-08 06:06:11 INFO: Got 100 lines from prob15input.txt.
2023-11-08 06:06:11 INFO: 15a. Lowest risk path: 447
2023-11-08 06:06:21 INFO: 15b. Lowest risk path: 2825


# Day 16:  Packet Decoder

## 16a.  What do you get if you add up the version numbers in all packets?

## 16b.  What do you get if you evaluate the expression?

In [18]:
filename = 'prob16input.txt'
lines = get_string_input(filename)


def read_literal_packet(version, bits):
    ''' Determine the value of literal packet.  bits length should be multiple of 5.
    '''
    retval = {'version':version, 'type':None, 'bitsRead':0, 'literal':None, 'lengthTypeId':None, 
              'versionSum':0, 'subPacketVals':[], 'packetVal':None}
    value = ''
    i = 0
    while 5*i < len(bits):
        retval['bitsRead'] += 5
        value += bits[1 + (5*i):5 + (5*i)]
        i += 1
        if bits[retval['bitsRead']-5] == '0':
            break

    return [5 * i, int(value,2)]


def read_packet(bits):
    ''' Recursively read bits and extract info into retval and return.
         bits = VVV100<literal values>  for literal packets (type = 4)
         bits = VVVTTTI<length param><packet info>   (type != 4)
    '''
    retval = {'version':None, 'type':None, 'bitsRead':0, 'literal':None, 'lengthTypeId':None, 
              'versionSum':0, 'subPacketVals':[], 'packetVal':None}

    retval['version'] = int(bits[0:3],2)  # first three bits provide version
    retval['type'] = int(bits[3:6],2)     # second three bits provide type
    retval['bitsRead'] += 6
    retval['versionSum'] += retval['version']
    
    if retval['type'] == 4:
        #logging.info("Reading literal packet")
        bits_read, value = read_literal_packet(retval['version'], bits[6:])
        retval['bitsRead'] += bits_read
        retval['literal'] = value
        retval['packetVal'] = value
    else:
        #logging.info("Reading non-literal packet")
        retval['lengthTypeId'] = bits[6]  
        retval['bitsRead'] += 1

        if retval['lengthTypeId'] == '0':
            total_length = int(bits[7:22],2)  #15 bits provide the length of packet
            retval['bitsRead'] += 15
            total_read = 0
            while total_read < total_length:
                # Read subpackets recursively.
                subPacketInfo = read_packet(bits[22+total_read:22+total_length])
                total_read += subPacketInfo['bitsRead']
                retval['versionSum'] += subPacketInfo['versionSum']
                retval['subPacketVals'].append(subPacketInfo['packetVal'])
            if total_read != total_length:
                logging.info("Error reading subpackets type 0.")
            else:
                retval['bitsRead'] += total_read

        else:
            number_of_subpackets = int(bits[7:18],2)  #11 bits provide number of subpackets
            retval['bitsRead'] += 11
            total_read = 0
            for sp in range(number_of_subpackets):
                subPacketInfo = read_packet(bits[18+total_read:])
                total_read += subPacketInfo['bitsRead']
                retval['versionSum'] += subPacketInfo['versionSum']
                retval['subPacketVals'].append(subPacketInfo['packetVal'])
            retval['bitsRead'] += total_read

    # Finished reading packet, set packet value based on type.
    if retval['type'] == 0:
        retval['packetVal'] = sum(retval['subPacketVals'])

    elif retval['type'] == 1:
        retval['packetVal'] = np.prod(retval['subPacketVals'])

    elif retval['type'] == 2:
        retval['packetVal'] = min(retval['subPacketVals'])

    elif retval['type'] == 3:
        retval['packetVal'] = max(retval['subPacketVals'])

    elif retval['type'] == 5:
        if len(retval['subPacketVals']) != 2:
            logging.error("Packet does not contain exactly two subpackets")
        elif retval['subPacketVals'][0] > retval['subPacketVals'][1]:
            retval['packetVal'] = 1
        else:
            retval['packetVal'] = 0

    elif retval['type'] == 6:
        if len(retval['subPacketVals']) != 2:
            logging.error("Packet does not contain exactly two subpackets")
        elif retval['subPacketVals'][0] < retval['subPacketVals'][1]:
            retval['packetVal'] = 1
        else:
            retval['packetVal'] = 0

    elif retval['type'] == 7:
        if len(retval['subPacketVals']) != 2:
            logging.error("Packet does not contain exactly two subpackets")
        elif retval['subPacketVals'][0] == retval['subPacketVals'][1]:
            retval['packetVal'] = 1
        else:
            retval['packetVal'] = 0

    return retval

# Get hexbits and convert to bit string.
hexbits = lines[0]
bit_dict = {'0':'0000','1':'0001','2':'0010','3':'0011','4':'0100','5':'0101','6':'0110','7':'0111',
            '8':'1000','9':'1001','A':'1010','B':'1011','C':'1100','D':'1101','E':'1110','F':'1111'}
bits = ''
for ch in hexbits:
    bits += bit_dict[ch]

# Read packet
solution = read_packet(bits)

logging.info("16a. Version Sum: {}".format(solution['versionSum'],))
logging.info("16a. Packet Value: {}".format(solution['packetVal'],))

2023-11-08 06:06:21 INFO: Got 1 lines from prob16input.txt.
2023-11-08 06:06:21 INFO: 16a. Version Sum: 979
2023-11-08 06:06:21 INFO: 16a. Packet Value: 277110354175


# Day 17:  Trick Shot

## 17a.  What is the highest y position it reaches on this trajectory?

## 17b.  How many distinct initial velocity values cause the probe to be within the target area after any step?

In [19]:
txMin, txMax, tyMin, tyMax = 60, 94, -171, -136  # puzzle input

def adjust_velocity(xVel, yVel):
    return min(xVel, abs(xVel-1)), yVel - 1

def get_next_position(pos, xVel, yVel):
    return [pos[0] + xVel, pos[1] + yVel]

def hit_target(pos, txMin, txMax, tyMin, tyMax):
    return (txMin <= pos[0] <= txMax and tyMin <= pos[1] <= tyMax)

# Find minimum velocity required to hit target
xMinVelocity, xMinPos = 0, 0
while xMinPos < txMin:
    xMinVelocity += 1
    xMinPos += xMinVelocity

# Loop over values 
yMax, hits = 0, 0
for xStartVel in range(xMinVelocity, txMax+1):
    for yStartVel in range(min(tyMin,tyMax), max(abs(tyMin),abs(tyMax))):
        pos, xVel, yVel, shotMax = [0,0], xStartVel, yStartVel, 0
        while pos[0] < txMax and pos[1] > tyMin:
            pos = get_next_position(pos, xVel, yVel)
            shotMax = max(shotMax, pos[1])
            xVel, yVel = adjust_velocity(xVel, yVel)
            if hit_target(pos, txMin, txMax, tyMin, tyMax):
                hits += 1
                yMax = max(yMax, shotMax)
                break
        
logging.info("17a. Max Height = {}".format(yMax,))
logging.info("17b. Trajectories that hit the target = {}".format(hits,))

2023-11-08 06:06:21 INFO: 17a. Max Height = 14535
2023-11-08 06:06:21 INFO: 17b. Trajectories that hit the target = 2270


# Day 18:  Snailfish

## 18a.  What is the magnitude of the final sum?

## 18b.  What is the largest magnitude of any sum of two different snailfish numbers from the homework assignment?

In [20]:
filename = 'prob18input.txt'
lines = get_string_input(filename)

# Vertex for binary tree
class Vertex:
    def __init__(self, val=None, left=None, right=None, depth=0):
        self.val = val
        self.left = left
        self.right = right
        self.depth = depth
        self.pad = '  ' * self.depth
        
    def show(self):
        if self.val != None:
            logging.info("{}{}  --- depth {}".format(self.pad, self.val, self.depth))
        else:
            self.left.show()
            logging.info("{}{}".format(self.pad,'-'))
            self.right.show()

# Functions for parsing input.            
def split_pair(pair_str):
    assert pair_str[0] == '['
    assert pair_str[-1] == ']'
    left_brackets = 0
    for i in range(len(pair_str)):
        if pair_str[i] == '[':
            left_brackets += 1
        elif pair_str[i] == ']':
            left_brackets -= 1
        elif pair_str[i] == ',' and left_brackets == 1:
            # split here
            left_str = pair_str[1:i]
            right_str = pair_str[i+1:-1]
            break
    return left_str, right_str

def is_pair_str(s):
    retval = False
    if s[0] == '[' and s[-1] == ']':
        retval = True
    return retval
                    
def get_snailfish_number(string, depth=0):
    
    retval = None
    if not is_pair_str(string):
        retval = Vertex(int(string),None,None,depth)
    else:
        left, right = split_pair(string)
        retval = Vertex(None, get_snailfish_number(left,depth+1), get_snailfish_number(right,depth+1),depth)
    return retval


# Functions for performing splits
def addLeftDown(valToAdd, sfNum):
    if sfNum.val != None:  sfNum.val += valToAdd
    else:                  addLeftDown(valToAdd, sfNum.left)
    
def addRightDown(valToAdd, sfNum):
    if sfNum.val != None:  sfNum.val += valToAdd
    else:                  addRightDown(valToAdd, sfNum.right)

def explode(sfNum):
    ''' Exploding sfNum if there are any pairs at depth == 4 (> 4 should not be possible)
        Return None if no explode performed.
    '''
    retval = None
    if sfNum.depth == 4 and sfNum.val == None:
        retval = [sfNum.left.val, sfNum.right.val]
        sfNum.left = None
        sfNum.right = None
        sfNum.val = 0
    elif sfNum.val == None:
        expLeft = explode(sfNum.left)
        if expLeft != None:
            retval = [expLeft[0], 0]
            if expLeft[1] != 0:
                addLeftDown(expLeft[1], sfNum.right)
        else:
            expRight = explode(sfNum.right)
            if expRight != None:
                retval = [0, expRight[1]]
                if expRight[0] != 0:
                    addRightDown(expRight[0], sfNum.left)
    else:
        pass # hit a number at an acceptable depth
    return retval


def split(sfNum):
    ''' Return the snailfish number sfNum with a split applied if possible.
         A value >= 10 is split into [floor(value/2), ceiling(value/2)]
         A maximum of one value is split in sfNum before returning.
         Recursively search sfNum for a value to split.
         Return False if no split performed.
    '''
    retval = False
    if sfNum.val == None:
        retval = split(sfNum.left)  # recursively look left...
        if retval == False:
            retval = split(sfNum.right) # .. then recursively look right.
    elif sfNum.val >= 10:
        retval = True
        sfNum.left = Vertex(math.floor(sfNum.val/2),None,None,sfNum.depth+1)
        sfNum.right = Vertex(math.ceil(sfNum.val/2),None,None,sfNum.depth+1)
        sfNum.val = None
    return retval


def reduce_snailfish(sfNum):
    ''' Reducing a snailfish number is done by
         1. Exploding all values until no explosions are left.
         2. Spliting any values > 10 if they exist.  If done, return to 1.
    '''
    unfinished = True
    while unfinished:
        sploded = []
        while sploded != None:
            sploded = explode(sfNum)
        unfinished = split(sfNum)  # returns sfNum if split found

def copy_snailfish(sfNum):
    ''' Create a copy of the snailfish number and return. '''
    retval = Vertex(sfNum.val, None, None, sfNum.depth)
    if sfNum.val == None:
        retval.left = copy_snailfish(sfNum.left)
        retval.right = copy_snailfish(sfNum.right)
    return retval
        

def increase_depth(sfNum):
    ''' Recursively increase the depth of each vertex in sfNum. '''
    sfNum.depth += 1
    if sfNum.val == None:
        increase_depth(sfNum.left)
        increase_depth(sfNum.right)
    

def add_sfNums(sf1, sf2):
    ''' Return the sum of the two snailfish numbers sf1 and sf2.
         sf1 + sf2 = [sf1, sf2] (followed by reduction)
    '''
    sf1_copy = copy_snailfish(sf1)
    sf2_copy = copy_snailfish(sf2)
    increase_depth(sf1_copy)
    increase_depth(sf2_copy)
    retval = Vertex(None, sf1_copy, sf2_copy, 0)
    reduce_snailfish(retval)
    return retval

def add_sfNumList(sfList):
    ''' Return the sum of the snailfish numbers in sfList.'''
    retval = None
    if len(sfList) > 1:
        retval = sfList[0]
        for s in sfList[1:]:
            retval = add_sfNums(retval, s)
    return retval

def get_magnitude(sfNum):
    ''' Return the magnitude of the snailfish number sfNum.'''
    retval = None
    if sfNum.val != None:
        retval = sfNum.val
    else:
        retval = 3 * get_magnitude(sfNum.left) + 2 * get_magnitude(sfNum.right)
    return retval

# Reading input into binary trees.
snailfish_nums = []
for l in lines:
    snailfish_nums.append(get_snailfish_number(l, 0))

sfResult = add_sfNumList(snailfish_nums)
mag = get_magnitude(sfResult)
logging.info('18a. Magnitude of list sum: {}'.format(mag,))

maxMag = 0
for s1, sf1 in enumerate(snailfish_nums):
    for s2, sf2 in enumerate(snailfish_nums):
        if s1 != s2:
            sfResult = add_sfNums(sf1, sf2)
            maxMag = max(maxMag, get_magnitude(sfResult))

logging.info('18b. Maximum magnitude: {}'.format(maxMag,))

2023-11-08 06:06:21 INFO: Got 100 lines from prob18input.txt.
2023-11-08 06:06:21 INFO: 18a. Magnitude of list sum: 4323
2023-11-08 06:06:23 INFO: 18b. Maximum magnitude: 4749


# Day 19:  Snailfish

## 19a.  How many beacons are there?

## 19b.  What is the largest Manhattan distance between any two scanners?

In [21]:
#   8-----7     z
#  /|    /|     |
# 4-----3 |     |
# | |   | |     |
# | 5---|-6     O-----y
# |/    |/     /
# 1-----2     x

class Scanner:
    def __init__(self, numId):
        self.id = numId
        self.beacons = []
        self.dists = dict()  # dists[d] = count of beacon pairs (b1, b2) with l1(b1, b2) = d
        self.loc = None
    
    def showBeacons(self, rotIndex=0):
        logging.info("Rotation {}".format(rotIndex,))
        for b in self.beacons:
            logging.info("beacon = {}".format(self.rotation(rotIndex, b[0], b[1], b[2]),))

    def rotation(self, index, x, y, z):
        retval = None
        if index == 0:    retval = [ x, y, z]
        elif index == 1:  retval = [ x, z,-y]
        elif index == 2:  retval = [ x,-y,-z]
        elif index == 3:  retval = [ x,-z, y]
        elif index == 4:  retval = [-z, y, x]
        elif index == 5:  retval = [-x, y,-z]
        elif index == 6:  retval = [ z, y,-x]
        elif index == 7:  retval = [ y,-x, z]
        elif index == 8:  retval = [-x,-y, z]
        elif index == 9:  retval = [-y, x, z]
        elif index == 10: retval = [ y, z, x]
        elif index == 11: retval = [ z,-x,-y]
        elif index == 12: retval = [-z, x,-y]
        elif index == 13: retval = [-z,-x, y]
        elif index == 14: retval = [-x, z, y]
        elif index == 15: retval = [-x,-z,-y]
        elif index == 16: retval = [-z,-y,-x]
        elif index == 17: retval = [ z,-y, x]
        elif index == 18: retval = [ y, x,-z]
        elif index == 19: retval = [-y,-x,-z]
        elif index == 20: retval = [-y, z,-x]
        elif index == 21: retval = [-y,-z, x]
        elif index == 22: retval = [ y,-z,-x]
        elif index == 23: retval = [ z, x, y]
        return retval

    def rotate_scanner(self, rotId):
        ''' Return a new scanner with beacons rotated. '''
        retval = Scanner(self.id)
        for b in self.beacons:
            newB = self.rotation(rotId, b[0], b[1], b[2])
            retval.beacons.append(newB)
        for d in self.dists.keys():
            retval.dists[d] = self.dists[d]
        return retval
    
    def get_l1_distances(self):
        ''' Populate self.dists
             self.dists[d] = number of pairs (b1, b2) in self.beacons
             with l1(b1, b2) = d
        '''
        for b1_index in range(len(self.beacons)-1):
            b1 = self.beacons[b1_index]
            for b2_index in range(b1_index+1, len(self.beacons)):
                b2 = self.beacons[b2_index]
                l1 = abs(b1[0] - b2[0]) + abs(b1[1] - b2[1]) + abs(b1[2] - b2[2])
                self.dists[l1] = self.dists.get(l1, 0) + 1

                
def get_dists_overlap(s1, s2):
    ''' Return the number of distances that are in both s1.dists and s2.dists'''
    retval = 0
    for d in s1.dists.keys():
        retval += min(s1.dists[d], s2.dists.get(d,0)) 
    return retval

def get_list_overlap(s1, s2, shift):
    ''' Return the number of values in the intersection of list s1 and shifted list s2. 
         s1 and s2 are sorted lists of integers.
    '''    
    retval = 0
    s1_index = 0
    s2_index = 0
    while s1_index < len(s1) and s2_index < len(s2):
        if s1[s1_index] < s2[s2_index] + shift:
            s1_index += 1
        elif s1[s1_index] > s2[s2_index] + shift:
            s2_index += 1
        else:
            retval += 1
            s1_index += 1
            s2_index += 1
    return retval

def find_overlap(s1, s2):
    ''' Rotate scanner 2 and shift to try to get overlap with scanner 1. 
        Return 4-tuple [rotation index, xShift, yShift, zShift] that produces overlap.
        Return None if no solution found.
        
        Note:  Function assumes that overlap found indepedently implies legitimate overlap
          which is not logically correct, but works in this case.
    '''
    retval = None   # [rot index for s2,  xShift for s2,  yShift for s2,  zShift for s2 ]
    x1Coords = sorted([b[0] for b in s1.beacons])
    y1Coords = sorted([b[1] for b in s1.beacons])
    z1Coords = sorted([b[2] for b in s1.beacons])

    for rotId in range(0,24):
        if retval != None:  break
        newS2 = s2.rotate_scanner(rotId)
        x2Coords = sorted([b[0] for b in newS2.beacons])  
        y2Coords = sorted([b[1] for b in newS2.beacons])  
        z2Coords = sorted([b[2] for b in newS2.beacons])  

        for xShift in range(-2000,2001):
            if retval != None:  break

            xOverlap = get_list_overlap(x1Coords, x2Coords, xShift)
            if xOverlap >= 12:
                for yShift in range(-2000,2001):
                    if retval != None:  break

                    yOverlap = get_list_overlap(y1Coords, y2Coords, yShift)
                    if yOverlap >= 12:

                        for zShift in range(-2000,2001):
                            zOverlap = get_list_overlap(z1Coords, z2Coords, zShift)
                            if zOverlap >= 12:
                                retval = [rotId, xShift, yShift, zShift]
                                break

    return retval


# Get input for problem
filename = 'prob19input.txt'
lines = get_string_input(filename)
scanner_re = re.compile('^--- scanner ([\d]+) ---\s*$')
garbage_re = re.compile('^\s*$')

scanners = []
cuScanner = None
for l in lines:
    m1 = scanner_re.match(l)
    m2 = garbage_re.match(l)
    if m1 != None:
        sNum = int(m1.groups(0)[0])
        curScanner = Scanner(sNum)
    elif m2 != None:
        curScanner.get_l1_distances()
        scanners.append(curScanner)
        curScanner = None
    else:
        beacon = [int(x) for x in l.split(',')]
        curScanner.beacons.append(beacon)
        
if curScanner != None:
    curScanner.get_l1_distances()
    scanners.append(curScanner)
    curScanner = None

    
# Find the "correct" location and orientation of the scanners.
found = [scanners[0]]
found[0].loc = [0, 0, 0]  # Assumed WLOG for simplicity.
scanners = scanners[1:]

found_index = 0
while len(scanners) > 0:
    s1 = found[found_index]  # Find all scanners that overlap with s1
    #logging.info("Looking around {} at {}".format(s1.id, s1.loc))
    not_found = []
    for s2 in scanners:
        overlap = None
        if get_dists_overlap(s1, s2) >= 66:  # simple pre test
            overlap = find_overlap(s1, s2)
        if overlap == None:
            not_found.append(s2)  # s2 does not overlap with s1, add to not_found
        else:
            #logging.info("Found {} next to {}".format(s2.id, s1.id))
            newS2 = s2.rotate_scanner(overlap[0])  # Rotate to get solution
            newS2.loc = [s1.loc[0] + overlap[1], s1.loc[1] + overlap[2], s1.loc[2] + overlap[3]]
            found.append(newS2)

    scanners = not_found
    found_index += 1

# Build one large Scanner called final
final = Scanner('Final')
for f in found:
    xloc, yloc, zloc = f.loc
    for b in f.beacons:
        newB = [b[0] + xloc, b[1] + yloc, b[2] + zloc]
        if newB not in final.beacons:
            final.beacons.append(newB)

logging.info("19a. Total beacons: {}".format(len(final.beacons),))

# Get l1 distances between each beacon.
dists = dict()
for s1_index, s1 in enumerate(found):
    for s2 in found[s1_index+1:]:
        l1 = abs(s1.loc[0] - s2.loc[0]) + abs(s1.loc[1] - s2.loc[1]) + abs(s1.loc[2] - s2.loc[2])
        dists[l1] = dists.get(l1, 0) + 1

logging.info("19b. Max l1 distance between scanners: {}".format(max(dists.keys()),))


2023-11-08 06:06:23 INFO: Got 1023 lines from prob19input.txt.
2023-11-08 06:06:39 INFO: 19a. Total beacons: 449
2023-11-08 06:06:39 INFO: 19b. Max l1 distance between scanners: 13128


# 20.  Trench Map

## 20a.  How many pixels are lit in the resulting image after two enhancements?

## 20b.  How many pixels are lit in the resulting image after fifty enhancements?

In [22]:
filename = 'prob20input.txt'
lines = get_string_input(filename)
key = lines[0]
A = lines[2:]

''' # Below is the key for the test image and the test image
key = '..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##\
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###\
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.\
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....\
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..\
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....\
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#'

A = ['#..#.',
'#....',
'##..#',
'..#..',
'..###']
'''

def shift_matrix(M, fill_char, border=2):
    retval = []
    N = len(M) + 2*border
    for row in range(N):
        col_str = ''
        if row < border or row >= N-border:
            col_str = fill_char * N
        else:
            col_str = fill_char*border + M[row-border] + fill_char*border
        retval.append(col_str)
    return retval

def get_neighborhood(i,j):
    return [ [i-1,j-1], [i-1,j], [i-1,j+1], [i,j-1], [i,j], [i,j+1],[i+1,j-1], [i+1,j], [i+1,j+1]]

def convert_pix_to_bits(pix_str):
    return ''.join([{'.':'0', '#':'1'}[ch] for ch in pix_str])

def get_enhanced_pixel(A, i, j, fill, key):
    pix_str = ''
    for r,s in get_neighborhood(i,j):
        ch = fill
        if 0 <= r < len(A) and 0 <= s < len(A):
            ch = A[r][s]
        pix_str += ch
    bit_str = convert_pix_to_bits(pix_str)
    value = int(bit_str,2)
    retval = key[value]
    return retval

def enhance_image(A, fill):
    A1 = shift_matrix(A, fill)
    #A1
    retval = []
    for i in range(len(A1)):
        row_str = ''
        for j in range(len(A1)):
            row_str += get_enhanced_pixel(A1, i, j, fill, key)
        retval.append(row_str)
    return retval

fill_chars = ['.','#']
for i in range(50):
    A = enhance_image(A, fill_chars[i%2])
    if i == 1:
        num_lit_after_2 = sum([row.count('#') for row in A])  #5483

num_lit_after_50 = sum([row.count('#') for row in A])  #5483
logging.info("20a. Number of lit pixels after two enhancements: {}".format(num_lit_after_2,))
logging.info("20b. Number of lit pixels after fifty enhancements: {}".format(num_lit_after_50,))

2023-11-08 06:06:39 INFO: Got 102 lines from prob20input.txt.
2023-11-08 06:06:48 INFO: 20a. Number of lit pixels after two enhancements: 5483
2023-11-08 06:06:48 INFO: 20b. Number of lit pixels after fifty enhancements: 18732


# 21.  Dirac Dice

## 21a.  What is the computed value when a player wins using the deterministic dice?

## 21b.  In how many universes does the player who wins the most win?

In [23]:
def get_rolls(last_value):
    return [last_value+1 %100, last_value+2 %100, last_value+3 %100]

def get_new_pos(pos, move):
    retval = (pos + move) % 10 
    if retval == 0:
        retval += 10
    return retval

# To speed up part two, only the sum and the number of ways to get
#  that sum is needed to count the universes.  This makes things
#  a little more managable and a little faster.
#1,1,1 = 3 -> 1
#1,1,2; 1,2,1; 2,1,1 = 4 -> 3
#1,1,3; 1,3,1, 3,1,1, 1,2,2, 2,1,2, 2,2,1 = 5 -> 6
#1,2,3; 1,3,2; 2,1,3, 2,3,1, 3,1,2; 3,2,1; 2,2,2 = 6 -> 7
#1,3,3; 3,1,3; 3,3,1; 2,2,3; 2,3,2; 3,2,2 = 7 -> 6
#2,3,3; 3,2,3; 3,3,2 = 8 -> 3
#3,3,3 = 9 -> 1
DIRAC_THREE_ROLE_SUM_DISTRIBUTION = [[3,1],[4,3],[5,6],[6,7],[7,6],[8,3],[9,1]]

def get_num_universe_wins(pos, score, curPlayer, depth=0):
    ''' Recursive depth first search function that counts the number of universes
         where each player wins given the current state.
        Note that the number of possible branches is reduce from 3*3*3 = 27 to the
         number of possible sums, 7, in the DIRAC_THREE_ROLE_SUM_DISTRIBUTION which
         was computed by hand above.
        Also note that pos and score are copies for 2 long integer lists.  This is
         assumed in the code below but may fail in other programming languages where
         this would need to be explicitly stated.
    
        Arguments:
            pos: 2 long list of ints:  pos[player] = current position of player
            score: 2 long list of ints:  score[player] = current score of player
            curPlayer: index of current player in {0, 1}
            depth: integer level in recursion (not needed but helpful in debugging)
    '''
    myIndent = ' '*depth
    #logging.info("{}get_wins({},{},{},{})".format(myIndent, pos, score, curPlayer, depth))
    retval = [0,0]  # wins for player1 and player2
    assert max(score) < maxScore
    #myKey = (pos[0], pos[1], score[0], score[1], curPlayer)
    for rollSum, count in DIRAC_THREE_ROLE_SUM_DISTRIBUTION:
        newPos, newScore = pos.copy(), score.copy()
        newPos[curPlayer] = get_new_pos(pos[curPlayer], rollSum)
        newScore[curPlayer] += newPos[curPlayer]
        if newScore[curPlayer] >= maxScore:
            #logging.info("{}Player{} gets {} wins for move {}".format(myIndent,curPlayer+1,count,rollSum))
            retval[curPlayer] += count   # curPlayer wins game in count different universes
        else:
            #logging.info("{}After move {}, counting wins".format(myIndent, rollSum))
            newWins = get_num_universe_wins(newPos, newScore, (curPlayer+1) % 2, depth+1)
            #logging.info("{}For   move {}, wins = {}*{}".format(myIndent,rollSum, newWins,count))
            retval[0] += newWins[0]*count
            retval[1] += newWins[1]*count

    #logging.info("{}Leaving, wins = {}".format(indent, retval))
    return retval


numRolls, lastValue, curPlayer = 0, 0, 0
maxScore = 1000
pos = [7,5]          # position of player 1 (index 0) and player 2 (index 1)
score = [0,0]        # score of player 1 (index 0) and player 2 (index 1)
while max(score) < maxScore:
    rolls = get_rolls(lastValue)
    lastValue = rolls[-1]
    numRolls += 3
    move = sum(rolls) % 10
    pos[curPlayer] = get_new_pos(pos[curPlayer], move)
    score[curPlayer] += pos[curPlayer]
    curPlayer = (curPlayer + 1) % 2    

logging.info("21a. Number of rolls times losing score: {}".format(numRolls * score[curPlayer],))


maxScore = 21
pos = [7,5]          # position of player 1 (index 0) and player 2 (index 1)
score = [0,0]        # score of player 1 (index 0) and player 2 (index 1)
wins = get_num_universe_wins(pos, score, 0)
winner = int(wins[0] <= wins[1]) # one-line hack to find index of max
logging.info("21b. Player {} wins in {} unverses.".format(winner, max(wins)))

# For part 2, this implementation takes about four minutes.  This could probably be improved
# by saving some additional state.  For example, have a dictionary where the keys are the
# 5-tuple state (player1Pos, player2Pos, player1Score, player2Score, curPlayer)
# and the values are the number of universes in which each player wins given the state.
# The size of the state space in part 2 should be no larger than 10*10*20*20*2 = 80000,
# which is well within the bounds of memory.  Saving and reusing work would seriously reduce the
# amount of repeated work.  I could not get such a solution to work before I lost interest.


2023-11-08 06:06:48 INFO: 21a. Number of rolls times losing score: 798147
2023-11-08 06:07:52 INFO: 21b. Player 0 wins in 809953813657517 unverses.


# 22.  Reactor Reboot

## 22a.  After the reboot, how many cubes are on in the initialization procedure region?

## 22b.  After the reboot, how many total cubes are on?

In [24]:
filename = 'prob22input.txt'
commands = get_string_input(filename)

'''commands = ['on x=-5..47,y=-31..22,z=-19..33',
'on x=-44..5,y=-27..21,z=-14..35',
'on x=-49..-1,y=-11..42,z=-10..38',
'on x=-20..34,y=-40..6,z=-44..1',
'off x=26..39,y=40..50,z=-2..11',
'on x=-41..5,y=-41..6,z=-36..8',
'off x=-43..-33,y=-45..-28,z=7..25',
'on x=-33..15,y=-32..19,z=-34..11',
'off x=35..47,y=-46..-34,z=-11..5',
'on x=-14..36,y=-6..44,z=-16..29',
'on x=-57795..-6158,y=29564..72030,z=20435..90618',
'on x=36731..105352,y=-21140..28532,z=16094..90401',
'on x=30999..107136,y=-53464..15513,z=8553..71215',
'on x=13528..83982,y=-99403..-27377,z=-24141..23996',
'on x=-72682..-12347,y=18159..111354,z=7391..80950',
'on x=-1060..80757,y=-65301..-20884,z=-103788..-16709',
'on x=-83015..-9461,y=-72160..-8347,z=-81239..-26856',
'on x=-52752..22273,y=-49450..9096,z=54442..119054',
'on x=-29982..40483,y=-108474..-28371,z=-24328..38471',
'on x=-4958..62750,y=40422..118853,z=-7672..65583',
'on x=55694..108686,y=-43367..46958,z=-26781..48729',
'on x=-98497..-18186,y=-63569..3412,z=1232..88485',
'on x=-726..56291,y=-62629..13224,z=18033..85226',
'on x=-110886..-34664,y=-81338..-8658,z=8914..63723',
'on x=-55829..24974,y=-16897..54165,z=-121762..-28058',
'on x=-65152..-11147,y=22489..91432,z=-58782..1780',
'on x=-120100..-32970,y=-46592..27473,z=-11695..61039',
'on x=-18631..37533,y=-124565..-50804,z=-35667..28308',
'on x=-57817..18248,y=49321..117703,z=5745..55881',
'on x=14781..98692,y=-1341..70827,z=15753..70151',
'on x=-34419..55919,y=-19626..40991,z=39015..114138',
'on x=-60785..11593,y=-56135..2999,z=-95368..-26915',
'on x=-32178..58085,y=17647..101866,z=-91405..-8878',
'on x=-53655..12091,y=50097..105568,z=-75335..-4862',
'on x=-111166..-40997,y=-71714..2688,z=5609..50954',
'on x=-16602..70118,y=-98693..-44401,z=5197..76897',
'on x=16383..101554,y=4615..83635,z=-44907..18747',
'off x=-95822..-15171,y=-19987..48940,z=10804..104439',
'on x=-89813..-14614,y=16069..88491,z=-3297..45228',
'on x=41075..99376,y=-20427..49978,z=-52012..13762',
'on x=-21330..50085,y=-17944..62733,z=-112280..-30197',
'on x=-16478..35915,y=36008..118594,z=-7885..47086',
'off x=-98156..-27851,y=-49952..43171,z=-99005..-8456',
'off x=2032..69770,y=-71013..4824,z=7471..94418',
'on x=43670..120875,y=-42068..12382,z=-24787..38892',
'off x=37514..111226,y=-45862..25743,z=-16714..54663',
'off x=25699..97951,y=-30668..59918,z=-15349..69697',
'off x=-44271..17935,y=-9516..60759,z=49131..112598',
'on x=-61695..-5813,y=40978..94975,z=8655..80240',
'off x=-101086..-9439,y=-7088..67543,z=33935..83858',
'off x=18020..114017,y=-48931..32606,z=21474..89843',
'off x=-77139..10506,y=-89994..-18797,z=-80..59318',
'off x=8476..79288,y=-75520..11602,z=-96624..-24783',
'on x=-47488..-1262,y=24338..100707,z=16292..72967',
'off x=-84341..13987,y=2429..92914,z=-90671..-1318',
'off x=-37810..49457,y=-71013..-7894,z=-105357..-13188',
'off x=-27365..46395,y=31009..98017,z=15428..76570',
'off x=-70369..-16548,y=22648..78696,z=-1892..86821',
'on x=-53470..21291,y=-120233..-33476,z=-44150..38147',
'off x=-93533..-4276,y=-16170..68771,z=-104985..-24507']'''
command_re = re.compile('^(\S+) x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)$')

cmds = []
for c in commands:
    m1 = command_re.match(c)
    if m1 == None:
        logging.error("Could not match: {}".format(c,))
    else:
        onOff, x1, x2, y1, y2, z1, z2 = m1.groups(0)
        x1, x2 = min(int(x1), int(x2)), max(int(x1), int(x2))
        y1, y2 = min(int(y1), int(y2)), max(int(y1), int(y2))
        z1, z2 = min(int(z1), int(z2)), max(int(z1), int(z2))
        cmds.append([onOff, x1, x2, y1, y2, z1, z2])
        #logging.info("{} {} {} {} {} {} {} {}".format(onOff, x1, x2, y1, y2, z1, z2, c))
logging.info("Finished.  Length of cmds = {}".format(len(cmds),))

# Part 1
onCubes = set()
for onOff, x1, x2, y1, y2, z1, z2 in cmds:
    coversX = (x1 < x2 and x1 <= 50 and x2 >= -50)
    coversY = (y1 < y2 and y1 <= 50 and y2 >= -50)
    coversZ = (z1 < z2 and z1 <= 50 and z2 >= -50)
    if coversX and coversY and coversZ:
        for x in range(x1, x2+1):
            for y in range(y1, y2+1):
                for z in range(z1, z2+1):
                    val = "{},{},{}".format(x,y,z)
                    if onOff == 'on':
                        onCubes.add(val)
                    else:
                        onCubes.discard(val)
logging.info("22a. Number of cubes turned on: {}".format(len(onCubes),))

# Part 2
# My approach takes advantage of the following fact:
#  The last command for a cube is the state in which it ends.
# Going through the list in reverse order, only the cubes that are not
#  already set need to be set.
# I save each cuboid that doesn't overlap with a previously saved cuboid.
# If the current cuboid overlaps with a saved cuboid, break the current
#  cuboid into smaller cuboids so that one is contained in the previous
#  cuboid and the others are not.  Then delete the current cuboid and consider
#  each smaller cuboid on its own.  Repeat until all sub-cuboids of the
#  original cuboid have been considered and then move on to the next.

# Once all cuboids have been considered, examine the saved cuboids and find
#  the sum of the volume of all "on" cuboids.

def is_chunk_dim_contained_in_solid_dim(chunkDim, solidDim):
    assert len(chunkDim) == len(solidDim) == 2
    return solidDim[0] <= chunkDim[0] and chunkDim[1] <= solidDim[1]

def is_chunk_contained_in_solid(chunk, solid):
    assert len(chunk) == len(solid) == 6
    if len(chunk) != len(solid) or len(chunk) != 6:
        logging.error("{} {}".format(chunk, solid))
    return is_chunk_dim_contained_in_solid_dim(chunk[0:2], solid[0:2]) and\
            is_chunk_dim_contained_in_solid_dim(chunk[2:4], solid[2:4]) and\
            is_chunk_dim_contained_in_solid_dim(chunk[4:6], solid[4:6])

def does_chunk_dim_overlap_solid_dim(chunkDim, solidDim):
    assert len(chunkDim) == len(solidDim) == 2
    return (solidDim[0] <= chunkDim[1] and chunkDim[0] <= solidDim[1])

def does_chunk_overlap_solid(chunk, solid):
    assert len(chunk) == len(solid) == 6
    if len(chunk) != len(solid) or len(chunk) != 6:
        logging.error("{} {}".format(chunk, solid))
    return does_chunk_dim_overlap_solid_dim(chunk[0:2], solid[0:2]) and\
            does_chunk_dim_overlap_solid_dim(chunk[2:4], solid[2:4]) and\
            does_chunk_dim_overlap_solid_dim(chunk[4:6], solid[4:6])

def split_chunk_dim_over_solid_dim(chunkDim, solidDim):
    assert len(chunkDim) == len(solidDim) == 2
    #logging.info("spilt_chunk {} over {}".format(chunkDim, solidDim))
    if chunkDim[1] < solidDim[0] or chunkDim[0] > solidDim[1]:
        # c0---c1  s0---s1   or  s0---s1  c0---c1
        retval = None
    elif chunkDim[0] == chunkDim[1]:
        # s0---c0=c1---s1  or  s0=c0=c1---s1  or  s0---c0=c1=s1  or  s0=c0=c1=s1
        retval = [[chunkDim[0], chunkDim[1]]]
    elif chunkDim[0] < solidDim[0] and solidDim[0] <= chunkDim[1] and chunkDim[1] <= solidDim[1]:
        # c0---c1=s0---s1  or  c0---s0---c1---s1  or  c0---s0---c1=s1
        retval = [[chunkDim[0], solidDim[0]-1], [solidDim[0], chunkDim[1]]]
    elif chunkDim[0] < solidDim[0] and solidDim[1] < chunkDim[1]:
        # c0---s0---s1---c1  or  c0---s0=s1---c1
        retval = [[chunkDim[0], solidDim[0]-1], [solidDim[0], solidDim[1]], [solidDim[1]+1,chunkDim[1]]]
    elif chunkDim[0] < solidDim[1] and chunkDim[1] <= solidDim[1]:
        # s0---c0---c1---s1  or  s0=c0---c1---s1  or  s0---c0---c1=s1  or  s0=c0---c1=s1
        retval = [[chunkDim[0], chunkDim[1]]]
    elif chunkDim[0] <= solidDim[1] and solidDim[1] < chunkDim[1]:
        # s0---c0---s1---c1  or  s0=c0---s1---c1  or  s0---c0=s1---c1  or  s0=c0=s1---c1
        retval = [[chunkDim[0], solidDim[1]], [solidDim[1]+1,chunkDim[1]]]
    else:
        logging.error("Failed in split_chunk_dim_over_solid_dim: {} {}".format(chunkDim, solidDim))
    return retval   


def break_chunk_over_solid(chunk, solid):
    retval = None
    if is_chunk_contained_in_solid(chunk, solid):
        #logging.info("Chunk contained in solid: {} in {}".format(chunk, solid))
        retval = "contained"
    elif not does_chunk_overlap_solid(chunk, solid):
        #logging.info("Disjoint:  {} in {}".format(chunk, solid))
        retval = "disjoint"
    else:
        xDims = split_chunk_dim_over_solid_dim(chunk[0:2],solid[0:2])
        yDims = split_chunk_dim_over_solid_dim(chunk[2:4],solid[2:4])
        zDims = split_chunk_dim_over_solid_dim(chunk[4:6],solid[4:6])
        #logging.info("Overlap:   {} in {}".format(chunk, solid))
        retval = [x + y + z for x in xDims for y in yDims for z in zDims]
        #logging.info("newChunks = {}".format(retval,))
    return retval


nonOverlappingSolids = []
for k1, c in enumerate(reversed(cmds)):
    if k1 % 20 == 0:
        logging.info("Beginning block {} with queue length {}".format(k1, len(nonOverlappingSolids)))
    onOff, block1 = c[0], c[1:]
    chunkQueue = [[block1,0]]
    while len(chunkQueue) > 0:
        chunkSplit = False
        chunk, nonOverlapId = chunkQueue.pop(0)
        #logging.info("New chunk: {}".format(chunk,))
        for _, solid in nonOverlappingSolids[nonOverlapId:]:
            newChunks = break_chunk_over_solid(chunk, solid)
            if newChunks == "contained":
                # chunk is contained in solid and 
                #  therefore chunk command overridden by solid
                # discard chunk entirely, do not add to chunkQueue
                chunkSplit = True   # Do not add chunk to nonOverlappingSolids
                break
            elif newChunks == "disjoint":
                # chunk and solid do not overlap
                # move chunk to next item in queue
                nonOverlapId += 1
            elif len(newChunks) > 0:
                # chunk and solid overlap
                # newChunks is chunk split into blocks that are either
                #  disjoint or contained in solid
                # add new chunks to queue and discard current chunk
                chunkSplit = True   # Do not add chunk to nonOverlappingSolids
                for newC in newChunks:
                    chunkQueue.append([newC, nonOverlapId])
                break

        # Add chunk to nonOverlapping blocks if it was not split
        if chunkSplit == False:
            nonOverlappingSolids.append([onOff, chunk])

        #logging.info("Length of Queue: {},  Length of nonOv: {}".format(len(chunkQueue),len(nonOverlappingSolids)))
logging.info("Finished all {} blocks.".format(len(cmds),))
logging.info("Number on non overlapping blocks: {}".format(len(nonOverlappingSolids),))

numOnCubes = 0
for onOff, b in nonOverlappingSolids:
    if onOff == "on":
        numOnCubes += (b[1] - b[0] + 1) * (b[3] - b[2] + 1) * (b[5] - b[4] + 1)
logging.info("22b. Number of cubes turned on: {}".format(numOnCubes,))
# This approach takes about five minutes to complete, however it is much
#  faster than my initial implementation that took an hour.



2023-11-08 06:07:52 INFO: Got 420 lines from prob22input.txt.
2023-11-08 06:07:52 INFO: Finished.  Length of cmds = 420
2023-11-08 06:07:53 INFO: 22a. Number of cubes turned on: 601104
2023-11-08 06:07:53 INFO: Beginning block 0 with queue length 0
2023-11-08 06:07:53 INFO: Beginning block 20 with queue length 38
2023-11-08 06:07:53 INFO: Beginning block 40 with queue length 103
2023-11-08 06:07:53 INFO: Beginning block 60 with queue length 329
2023-11-08 06:07:53 INFO: Beginning block 80 with queue length 539
2023-11-08 06:07:54 INFO: Beginning block 100 with queue length 748
2023-11-08 06:07:54 INFO: Beginning block 120 with queue length 1072
2023-11-08 06:07:55 INFO: Beginning block 140 with queue length 1663
2023-11-08 06:07:56 INFO: Beginning block 160 with queue length 2042
2023-11-08 06:07:57 INFO: Beginning block 180 with queue length 2532
2023-11-08 06:07:59 INFO: Beginning block 200 with queue length 3106
2023-11-08 06:08:02 INFO: Beginning block 220 with queue length 4004
20

# 23.  Amphipod

## 23a.  What is the least energy required to organize the amphipods?

## 23b.  What is the least energy required to organize all the amphipods?

In [25]:

#1st    
class Edge:
    ''' Class to hold edges in a simple undirected graph. '''
    def __init__(self, a, b):
        self.u = min(a,b)
        self.v = max(a,b)

class Graph:
    ''' Class to hold a graph.
         V is a dict with keys are vertices (str) and values are neighborhoods ([v1, v2, ...])
         E is a list of Edges from the edge class
         order is the number vertices
         size it the number of edges
    '''
    def __init__(self):
        self.V = dict()  # V[v] = Neighbors
        self.E = []
        self.size = 0
        self.order = 0

    def isVertex(self, v):
        ''' Boolean to test if string v is already a vertex in graph. '''
        retval = False
        if self.V.get(v, None) != None:
            retval = True
        return retval

    def addVertex(self, v):
        ''' Add string v to dict of vertices; initialize with empty neighborhood. '''
        self.V[v] = []
        self.order += 1
    
    def addEdge(self, a, b):
        ''' Add the edge (a,b) to the graph.  That is, add the vertices a and b to
             the graph if needed, create Edge(a,b) and add it to E, and and a and b
             to the neighborhoods of b and a, respectively.
        '''
        if not self.isVertex(a):
            self.addVertex(a)
        if not self.isVertex(b):
            self.addVertex(b)
        self.E.append(Edge(a,b))
        self.V[a].append(b)
        self.V[b].append(a)
        self.size += 1
        
    def showGraph(self):
        ''' Output graph. '''
        logging.info("Graph.V = {}".format(self.V.keys(),))
        logging.info("Graph.E = {}".format([[e.u,e.v] for e in self.E],))
        
def create_burrow_graph():
    ''' Create and return the graph for part A of the problem.  See below. '''
    # locations         # Example start     # Problem
    # #12345678901#     # #...........#     # #...........#
    # ###2#4#6#8###     # ###B#C#B#D###     # ###A#C#B#C###
    #   #3#5#7#9#       #   #A#D#C#A#       #   #D#A#D#B#  

    retval = Graph()
    retval.addEdge(1,2)
    retval.addEdge(2,3)
    retval.addEdge(3,4)
    retval.addEdge(4,5)
    retval.addEdge(5,6)
    retval.addEdge(6,7)
    retval.addEdge(7,8)
    retval.addEdge(8,9)
    retval.addEdge(9,10)
    retval.addEdge(10,11)
    retval.addEdge(3,12)
    retval.addEdge(12,13)
    retval.addEdge(5,14)
    retval.addEdge(14,15)
    retval.addEdge(7,16)
    retval.addEdge(16,17)
    retval.addEdge(9,18)
    retval.addEdge(18,19)
    #retval.showGraph()
    return retval

def create_larger_burrow_graph():
    ''' Create and return the graph for part B of the problem.  See below. '''
    # Locations:       # Example start:   # Problem start
    # #12345678901#    # #...........#    # #...........#
    # ###2#6#0#4###    # ###B#C#B#D###    # ###A#C#B#C###
    #   #3#7#1#5#      #   #D#C#B#A#      #   #D#C#B#A#  
    #   #4#8#2#6#      #   #D#B#A#C#      #   #D#B#A#C#  
    #   #5#9#3#7#      #   #A#D#C#A#      #   #D#A#D#B#  

    retval = Graph()
    retval.addEdge(1,2)
    retval.addEdge(2,3)
    retval.addEdge(3,4)
    retval.addEdge(4,5)
    retval.addEdge(5,6)
    retval.addEdge(6,7)
    retval.addEdge(7,8)
    retval.addEdge(8,9)
    retval.addEdge(9,10)
    retval.addEdge(10,11)
    retval.addEdge(3,12)
    retval.addEdge(12,13)
    retval.addEdge(13,14)
    retval.addEdge(14,15)
    retval.addEdge(5,16)
    retval.addEdge(16,17)
    retval.addEdge(17,18)
    retval.addEdge(18,19)
    retval.addEdge(7,20)
    retval.addEdge(20,21)
    retval.addEdge(21,22)
    retval.addEdge(22,23)
    retval.addEdge(9,24)
    retval.addEdge(24,25)
    retval.addEdge(25,26)
    retval.addEdge(26,27)
    #retval.showGraph()
    return retval

class StateA:
    ''' Class to hold a state in the search algorithm.  A state consists of
         1) The current positions in the graph of each piece
         2) The moves applied to the initial state that created current state
         3) The energy required to move from initial state to current state give moves
    '''
    def __init__(self, pos):
        self.pos = pos  # dict():  pos[piece] = position of piece
        self.other = {'A1':'A2','A2':'A1','B1':'B2','B2':'B1','C1':'C2','C2':'C1','D1':'D2','D2':'D1'}
        self.moves  = []  # [ [piece, new_location, distance_for_move], ... ]
        self.energy = 0

    def copy(self):
        retval = StateA(self.pos.copy())
        retval.other = self.other
        retval.moves = self.moves[:]
        retval.energy = self.energy
        return retval

    def get_other_piece_pos(self, p):
        ''' Return the position of the other similar piece. '''
        return self.pos[ self.other[p] ]

    def get_all_moves_for_pos(self, pos):
        ''' Return a list of [location, dist] values that indicate how
             many moves (dist) to move from pos to location.
        '''
        occupied = set(self.pos.values())
        # Initialize retval with moves of distance 1
        retval = [[v,1] for v in G.V[pos] if v not in occupied]
        used = set([m[0] for m in retval]) # locations already added to moves
        index = 0
        while index < len(retval):
            old_pos, old_dist = retval[index]  
            # Get all moves of distance = old_dist+1 
            new_moves = [[v, old_dist+1] for v in G.V[old_pos] if v not in occupied and v not in used]
            for m in new_moves:
                used.add(m[0])
            retval += new_moves
            index += 1
        return retval

    def get_possible_moves(self, G):
        ''' Return a list of [piece, location, distance] for all reasonable moves,
             given current state, of piece to location with shortest path having distance.
        '''
        retval = []
        for piece, pos in self.pos.items():
            # Consider all moves for piece but only save if passes all tests
            for newPos, dist in self.get_all_moves_for_pos(pos):
                if newPos in [3,5,7,9]:
                    pass #logging.info("Should not move 0: {} {} {}".format(piece, pos, m))
                elif newPos == 13 and piece not in ['A1','A2']:
                    pass #logging.info("Should not move 1: {} {} {}".format(piece, pos, m))
                elif newPos == 12 and (piece not in ['A1','A2'] or self.get_other_piece_pos(piece) != 13):
                    pass #logging.info("Should not move 2: {} {} {}".format(piece, pos, m))
                elif newPos == 15 and piece not in ['B1','B2']:
                    pass #logging.info("Should not move 3: {} {} {}".format(piece, pos, m))
                elif newPos == 14 and (piece not in ['B1','B2'] or self.get_other_piece_pos(piece) != 15):
                    pass #logging.info("Should not move 4: {} {} {}".format(piece, pos, m))
                elif newPos == 17 and piece not in ['C1','C2']:
                    pass #logging.info("Should not move 5: {} {} {}".format(piece, pos, m))
                elif newPos == 16 and (piece not in ['C1','C2'] or self.get_other_piece_pos(piece) != 17):
                    pass #logging.info("Should not move 6: {} {} {}".format(piece, pos, m))
                elif newPos == 19 and piece not in ['D1','D2']:
                    pass #logging.info("Should not move 7: {} {} {}".format(piece, pos, m))
                elif newPos == 18 and (piece not in ['D1','D2'] or self.get_other_piece_pos(piece) != 19):
                    pass #logging.info("Should not move 8: {} {} {}".format(piece, pos, m))
                elif newPos in [1,2,4,6,8,10,11] and pos in [1,2,4,6,8,10,11]:
                    pass #logging.info("Should not move 9: {} {} {}".format(piece, pos, m))
                elif piece in ['A1','A2'] and pos == 13:
                    pass #logging.info("Will not move 10: {} {} {}".format(piece, pos, m))
                elif piece in ['A1','A2'] and pos == 12 and self.get_other_piece_pos(piece) == 13:
                    pass #logging.info("Will not move 11: {} {} {}".format(piece, pos, m))
                elif piece in ['B1','B2'] and pos == 15:
                    pass #logging.info("Will not move 12: {} {} {}".format(piece, pos, m))
                elif piece in ['B1','B2'] and pos == 14 and self.get_other_piece_pos(piece) == 15:
                    pass #logging.info("Will not move 13: {} {} {}".format(piece, pos, m))
                elif piece in ['C1','C2'] and pos == 17:
                    pass #logging.info("Will not move 14: {} {} {}".format(piece, pos, m))
                elif piece in ['C1','C2'] and pos == 16 and self.get_other_piece_pos(piece) == 17:
                    pass #logging.info("Will not move 15: {} {} {}".format(piece, pos, m))
                elif piece in ['D1','D2'] and pos == 19:
                    pass #logging.info("Will not move 16: {} {} {}".format(piece, pos, m))
                elif piece in ['D1','D2'] and pos == 18 and self.get_other_piece_pos(piece) == 19:
                    pass #logging.info("Will not move 17: {} {} {}".format(piece, pos, m))
                else:
                    pass #logging.info("Move allowed A: {} {} {}".format(piece, pos, m))
                    retval.append([piece, newPos, dist])
                    
        #logging.info("Finished evaluating moves for {}".format(retval))
        return retval
        
    def showState(self):
        ''' A debugging function that outputs the state. '''
        logging.info("State...")
        logging.info("Positions: {}".format(self.pos,))
        logging.info("Moves: {}".format(self.moves,))
        logging.info("Energy: {}".format(self.energy,))
        
    def get_state_key(self):
        ''' Return a unique string that describes the current state locations.
             This function generalizes so states are considered equivalent if
             the set of pieces of the same type (e.g. A1 and A2) can be permuted.
            Note: This is called frequently, so this is faster if the list 
             comprehension is not done for each type of piece and the pieces
             are listed separately... however this generalizes nicer and looks
             prettier so it is worth the couple of extra seconds.
        '''
        posA = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'A'])
        posB = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'B'])
        posC = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'C'])
        posD = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'D'])
        return "{}-{}-{}-{}-{}-{}-{}-{}".format(*(posA + posB + posC + posD))
    
    def is_finished(self):
        ''' Boolean to test if state is a finished state for part A. '''
        retval = False
        if self.pos['A1'] in [12,13] and self.pos['A2'] in [12,13] and\
           self.pos['B1'] in [14,15] and self.pos['B2'] in [14,15] and\
           self.pos['C1'] in [16,17] and self.pos['C2'] in [16,17] and\
           self.pos['D1'] in [18,19] and self.pos['D2'] in [18,19]:
            retval = True
        return retval

class StateB(StateA):
    ''' Similar to StateA but had to add/redefine some functions to work
         with additional pieces and different board shape.
    '''
    def copy(self):
        retval = StateB(self.pos.copy())
        retval.moves = self.moves[:]
        retval.energy = self.energy
        return retval

    def get_other_piece_pos(self, p):
        ''' Return a list of all other positions of similar pieces. '''
        retval = []   # Now an array
        for k in self.pos.keys():
            if k[0] == p[0] and k != p:
                retval.append(self.pos[k])
        return retval

    def is_situated_in_room(self, piece, pos):
        ''' Boolean to indicate if piece is in a target location. '''
        retval = False
        if pos in [12,13,14,15] and piece[0] == 'A':
            other_piece_pos = self.get_other_piece_pos(piece)
            retval = all([i in other_piece_pos for i in range(pos+1,16)])
        elif pos in [16,17,18,19] and piece[0] == 'B':
            other_piece_pos = self.get_other_piece_pos(piece)
            retval = all([i in other_piece_pos for i in range(pos+1,20)])
        elif pos in [20,21,22,23] and piece[0] == 'C':
            other_piece_pos = self.get_other_piece_pos(piece)
            retval = all([i in other_piece_pos for i in range(pos+1,24)])
        elif pos in [24,25,26,27] and piece[0] == 'D':
            other_piece_pos = self.get_other_piece_pos(piece)
            retval = all([i in other_piece_pos for i in range(pos+1,28)])
        return retval

    def get_possible_moves(self, G):
        ''' Return a list of [piece, location, distance] for all reasonable moves,
             given current state, of piece to location with shortest path having distance.
        '''
        retval = []
        for piece, pos in self.pos.items():
            # Consider all moves for piece but only save if passes all tests
            for m in self.get_all_moves_for_pos(pos):
                if m[0] in [3,5,7,9]:
                    pass #logging.info("Cannot move 0: {} {} {}".format(piece, pos, m))
                elif 12 <= m[0] < 28 and not self.is_situated_in_room(piece, m[0]):
                    pass
                elif m[0] in [1,2,4,6,8,10,11] and pos in [1,2,4,6,8,10,11]:
                    pass #logging.info("Cannot move 9: {} {} {}".format(piece, pos, m))
                elif self.is_situated_in_room(piece, pos):
                    pass #logging.info("Will not move 10: {} {} {}".format(piece, pos, m))
                else:
                    pass #logging.info("Move allowed A: {} {} {}".format(piece, pos, m))
                    retval.append([piece, m[0], m[1]])
        return retval

    def get_state_key(self):
        posA = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'A'])
        posB = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'B'])
        posC = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'C'])
        posD = sorted([self.pos[x] for x in self.pos.keys() if x[0] == 'D'])
        return "{}-{}-{}-{}-{}-{}-{}-{}-{}-{}-{}-{}-{}-{}-{}-{}".format(*(posA + posB + posC + posD))

    def is_finished(self):
        retval = False
        if all([self.pos[p] in [12,13,14,15] for p in ['A1','A2','A3','A4']]) and \
           all([self.pos[p] in [16,17,18,19] for p in ['B1','B2','B3','B4']]) and \
           all([self.pos[p] in [20,21,22,23] for p in ['C1','C2','C3','C4']]) and \
           all([self.pos[p] in [24,25,26,27] for p in ['D1','D2','D3','D4']]):
            retval = True
        return retval


for problem in ['a', 'b']:
    # PartA
    G = create_burrow_graph()
    costs = {'A':1,'B':10,'C':100,'D':1000}
    #state = StateA({'A1':13, 'A2':19, 'B1':12, 'B2':16, 'C1':14, 'C2':17, 'D1':15, 'D2':18})  # Test A
    state = StateA({'A1':12, 'A2':15, 'B1':16, 'B2':19, 'C1':14, 'C2':18, 'D1':13, 'D2':17})   # Problem A
    if problem == 'b':
        logging.info("Switch to b")
        G = create_larger_burrow_graph()
        #state = StateB({'A1':15,'A2':22,'A3':25,'A4':27,
        #                'B1':12,'B2':18,'B3':20,'B4':21,
        #                'C1':16,'C2':17,'C3':23,'C4':26,
        #                'D1':13,'D2':14,'D3':19,'D4':24}) # This is the example
        state = StateB({'A1':12,'A2':19,'A3':22,'A4':25,
                        'B1':18,'B2':20,'B3':21,'B4':27,
                        'C1':16,'C2':17,'C3':24,'C4':26,
                        'D1':13,'D2':14,'D3':15,'D4':23})  # This is the problem

    states_found = dict()  # states_found[state] = energy to get to state
    states_found[state.get_state_key()] = state.energy
    states = [state]   # Initialize Queue
    movesMade, sort_count = 0, 0  # not required, but informative
    next_state_energy, min_energy_added = False, False

    # loop through a states queue, popping the lowest energy state and making all
    # possible (reasonable) moves and adding those to the queue until solution found.        
    while len(states) > 0:
        s = states.pop(0)     # Get next state for this iteration
        if s.is_finished() == True:
            s.showState()
            logging.info("Finished!")
            break
        next_state_energy = None if len(states) == 0 else states[0].energy

        # Make each possible move and save result as newState, append newState to queue.
        for piece, newPos, dist in s.get_possible_moves(G):
            newState = s.copy()                           # create copy of old state
            newState.pos[piece] = newPos                  # move piece to new location
            newState.energy += (costs[piece[0]] * dist)   # add energy for cost of move
            newState.moves.append([piece, newPos, dist])  # append new move to moves list
            # Check if state already found (using nsKey); add if not; update if new is better
            nsKey = newState.get_state_key()
            nsPrevVal = states_found.get(nsKey,0)
            if (nsPrevVal == 0) or (0 < newState.energy < nsPrevVal):
                states.append(newState)
                states_found[nsKey] = newState.energy
                min_energy_added = newState.energy if min_energy_added == None else min(min_energy_added,newState.energy)
            else:
                pass #logging.info("Previously seen state: {}, {}, {}".format(nsKey, nsPrevVal, newState.energy))
            
        # check if any states added that require sort before next iteration
        if (next_state_energy == None) or (min_energy_added != None and min_energy_added < next_state_energy):
            states = sorted(states, key=lambda x: x.energy)
            sort_count += 1  # Not required, but informative
            min_energy_added = None
        
        movesMade += 1        # Not required, but informative
        if movesMade % 10000 == 0:
            logging.info("Move {}, queueLength = {}, Energy = {}, Sorts = {}".format(movesMade, len(states), s.energy, sort_count))
    logging.info("23{}. The minimal energy to get to a solution: {}".format(problem, s.energy))


2023-11-08 06:09:35 INFO: Move 10000, queueLength = 13357, Energy = 3463, Sorts = 200
2023-11-08 06:09:36 INFO: Move 20000, queueLength = 12198, Energy = 4210, Sorts = 285
2023-11-08 06:09:36 INFO: Move 30000, queueLength = 14881, Energy = 6007, Sorts = 473
2023-11-08 06:09:37 INFO: Move 40000, queueLength = 13064, Energy = 7063, Sorts = 624
2023-11-08 06:09:38 INFO: Move 50000, queueLength = 11551, Energy = 8159, Sorts = 813
2023-11-08 06:09:39 INFO: Move 60000, queueLength = 9353, Energy = 9505, Sorts = 974
2023-11-08 06:09:40 INFO: Move 70000, queueLength = 9942, Energy = 10790, Sorts = 1142
2023-11-08 06:09:40 INFO: Move 80000, queueLength = 6664, Energy = 13090, Sorts = 1379
2023-11-08 06:09:41 INFO: Move 90000, queueLength = 2829, Energy = 16199, Sorts = 1631
2023-11-08 06:09:41 INFO: State...
2023-11-08 06:09:41 INFO: Positions: {'A1': 13, 'A2': 12, 'B1': 15, 'B2': 14, 'C1': 16, 'C2': 17, 'D1': 18, 'D2': 19}
2023-11-08 06:09:41 INFO: Moves: [['A1', 2, 2], ['B1', 4, 4], ['C2', 6,

# 24.  Arithmetic Logic Unit

## 24a.  What is the largest input accepted by MONAD?

## 24b.  What is the smallest input accepted by MONAD?

In [26]:
filename = 'prob24input.txt'
commands = get_string_input(filename)

inp_re = re.compile('^inp ([wxyz])\s*$')
add_re = re.compile('^add ([wxyz]) ([wxyz\d\-]+)\s*$')
mul_re = re.compile('^mul ([wxyz]) ([wxyz\d\-]+)\s*$')
div_re = re.compile('^div ([wxyz]) ([wxyz\d\-]+)\s*$')
mod_re = re.compile('^mod ([wxyz]) ([wxyz\d]+)\s*$')
eql_re = re.compile('^eql ([wxyz]) ([wxyz\d\-]+)\s*$')

coms = []
for c in commands:
    mInp = inp_re.match(c)
    mAdd = add_re.match(c)
    mMul = mul_re.match(c)
    mDiv = div_re.match(c)
    mMod = mod_re.match(c)
    mEql = eql_re.match(c)
    if mInp != None:
        coms.append(['inp', mInp.groups(1)[0]])
    elif mAdd != None:
        coms.append(['add', mAdd.groups(1)[0], mAdd.groups(1)[1]])
    elif mMul != None:
        coms.append(['mul', mMul.groups(1)[0], mMul.groups(1)[1]])
    elif mDiv != None:
        coms.append(['div', mDiv.groups(1)[0], mDiv.groups(1)[1]])
    elif mMod != None:
        coms.append(['mod', mMod.groups(1)[0], mMod.groups(1)[1]])
    elif mEql != None:
        coms.append(['eql', mEql.groups(1)[0], mEql.groups(1)[1]])
    else:
        logging.error("Garbage Input: '{}'".format(c,))    

# Careful examination of the input file reveals that every set of 18 lines
#  are very similar.  There are three exceptions which are the values in
#  lines 5, 6, and 16 (labeled val1, val2, val3).  Instead of performing
#  each of the 18 lines sequentially, it was easier to create a function
#  that performs all 18 lines with the three input values.  The logic in
#  the function returned by get_round_func is equivalent.

def get_round_func(val1, val2, val3):
    ''' Return a function that will perform the "round" of 18 commands where
         the three inputs, val1, val2, and val3, are integers.
        Technically, the values for x and y are not needed since they are
         multiplied by 0 before being used for anything.
    '''
    # logging.info("{}, {}, {}".format(val1, val2, val3))
    assert val1 == 1 or val1 == 26
    assert val3 > 0
    def retval(state):
        ''' Generate all 9 new states for the round and perform the operations.
             Return a list of the 9 new states generated from the input state.
        '''
        retvalretval = []
        for w in range(1,10):
            s = state.copy()
            s['w'] = w
            if s['w'] == (s['z'] % 26) + val2:
                s['x'] = 0
                s['y'] = 0
                s['z'] = s['z'] // val1
            else:
                s['x'] = 1
                s['y'] = s['w'] + val3
                s['z'] = 26*(s['z']//val1) + s['w'] + val3
            s['step'] += 18
            s['input'] = s['input'] * 10 + w
            retvalretval.append(s)
        return retvalretval

    return retval


# Create cFuncs
# Break the input into rounds where each round is a single round function that takes three inputs.
cFuncs = []
skip = 0
for i, c in enumerate(coms):
    if c[0] == 'inp' and c[1] == 'w' and i < len(coms)-17:
        if coms[i+1][0]   == 'mul' and coms[i+1][1]  == 'x' and coms[i+1][2]  == '0' and \
            coms[i+2][0]  == 'add' and coms[i+2][1]  == 'x' and coms[i+2][2]  == 'z' and \
            coms[i+3][0]  == 'mod' and coms[i+3][1]  == 'x' and coms[i+3][2]  == '26' and \
            coms[i+4][0]  == 'div' and coms[i+4][1]  == 'z' and \
            coms[i+5][0]  == 'add' and coms[i+5][1]  == 'x' and \
            coms[i+6][0]  == 'eql' and coms[i+6][1]  == 'x' and coms[i+6][2]  == 'w' and \
            coms[i+7][0]  == 'eql' and coms[i+7][1]  == 'x' and coms[i+7][2]  == '0' and \
            coms[i+8][0]  == 'mul' and coms[i+8][1]  == 'y' and coms[i+8][2]  == '0' and \
            coms[i+9][0]  == 'add' and coms[i+9][1]  == 'y' and coms[i+9][2]  == '25' and \
            coms[i+10][0] == 'mul' and coms[i+10][1] == 'y' and coms[i+10][2] == 'x' and \
            coms[i+11][0] == 'add' and coms[i+11][1] == 'y' and coms[i+11][2] == '1' and \
            coms[i+12][0] == 'mul' and coms[i+12][1] == 'z' and coms[i+12][2] == 'y' and \
            coms[i+13][0] == 'mul' and coms[i+13][1] == 'y' and coms[i+13][2] == '0' and \
            coms[i+14][0] == 'add' and coms[i+14][1] == 'y' and coms[i+14][2] == 'w' and \
            coms[i+15][0] == 'add' and coms[i+15][1] == 'y' and \
            coms[i+16][0] == 'mul' and coms[i+16][1] == 'y' and coms[i+16][2] == 'x' and \
            coms[i+17][0] == 'add' and coms[i+17][1] == 'z' and coms[i+17][2] == 'y':
            cFuncs.append(get_round_func( int(coms[i+4][2]), int(coms[i+5][2]), int(coms[i+15][2]) ))
            skip = 18
    if skip > 0:
        skip -= 1

# Build a bound for z after each step (see analysis)
# Let 
#  z_i (the value of z at the end of round i) 
#  a_i (the value of at the end of "div z" line in round i) 
#  b_i (the value of at the end of "add x" line in round i) 
#  c_i (the value of at the end of "add y" line in round i) 
# Examining the round functions and the input shows that each z_i is the combination of positive values.
# Further examination shows that each z_{i-1} < a_i(z_i + 1) since:
#   z_i = 26(z_{i-1}//a_i) + w_i + c_i
#       > 26(z_{i-1}//a_i)     (since w_i > 0 and c_i > 0)
#       >= z_{i-1}//a_i        (since z_{i-1}//a_i >= 0)   
# And
#   z_i == z_{i-1} // a_i 
#   z_i + 1 > z_{i-1} / a_i
#   z_{i-1} < a_i(z_i + 1)    *** bound equation ***
# So in both cases the result is true.  Therefore, using a_i and the fact that z_{14} must be 0
#  a bound can be iteratively determined using the bound equation above.

a = [1, 1, 1, 1, 26, 1, 1, 26, 1, 26, 26, 26, 26, 26]  # hard coded from input
zBound = [0]
for mya in reversed(a):
    zBound.append(mya*(zBound[-1] + 1))
zBounds = dict([x for x in enumerate(reversed(zBound))])

# Solve problem a with the max() and problem b with the min()
for prob, minmax in zip(['a', 'b'],[max, min]):
    queue = [{'w':0, 'x':0, 'y':0, 'z':0, 'step':-1, 'input':0}]
    cols, disc = 0, 0
    # For each round (18 lines of input)
    for round, cf in enumerate(cFuncs):
        step = queue[0]['step']
        roundStates = dict()
        # For each state in queue, generate new states for next round
        for state in queue:
            for newState in cf(state):
                if newState['z'] > zBounds[round+1]:
                    disc += 1
                    continue # Discard newState as z value exceeds bound (see above)
                stateKey = newState['z']   # After each round, only save the minmax input associated with each z value
                oldState = roundStates.get(stateKey, None)
                if oldState == None:
                    roundStates[stateKey] = newState
                elif minmax([oldState['input'], newState['input']]) == newState['input']:
                    cols += 1   # Collision, use preferred newState
                    roundStates[stateKey] = newState
                else:
                    cols += 1   # Collision, use preferred oldState
        queue = sorted(roundStates.values(), key=lambda x: x['input'])
        logging.info("After round {}, Queue: {}, Collisions: {}, Discarded: {}".format(round+1, len(queue), cols, disc))
    assert len(queue) == 1  # There should only be one state
    logging.info("24{}. The {} solution is {}".format(prob, minmax.__name__, queue[0]['input']))



2023-11-08 06:09:50 INFO: Got 252 lines from prob24input.txt.
2023-11-08 06:09:50 INFO: After round 1, Queue: 9, Collisions: 0, Discarded: 0
2023-11-08 06:09:50 INFO: After round 2, Queue: 81, Collisions: 0, Discarded: 0
2023-11-08 06:09:50 INFO: After round 3, Queue: 729, Collisions: 0, Discarded: 0
2023-11-08 06:09:50 INFO: After round 4, Queue: 6561, Collisions: 0, Discarded: 0
2023-11-08 06:09:50 INFO: After round 5, Queue: 7290, Collisions: 51759, Discarded: 0
2023-11-08 06:09:50 INFO: After round 6, Queue: 65610, Collisions: 51759, Discarded: 0
2023-11-08 06:09:51 INFO: After round 7, Queue: 590490, Collisions: 51759, Discarded: 0
2023-11-08 06:09:55 INFO: After round 8, Queue: 114453, Collisions: 763992, Discarded: 4487724
2023-11-08 06:09:56 INFO: After round 9, Queue: 59049, Collisions: 763992, Discarded: 5458752
2023-11-08 06:09:56 INFO: After round 10, Queue: 6561, Collisions: 803358, Discarded: 5944266
2023-11-08 06:09:56 INFO: After round 11, Queue: 729, Collisions: 803358

# 25.  Sea Cucumber

## 25a.  What is the first step on which no sea cucumbers move?

## 25b.  Free Star!

In [27]:
lines = ['v...>>.vv>',
'.vv>>.vv..',
'>>.>v>...v',
'>>v>>.>.v.',
'v>v.vv.v..',
'>.>>..v...',
'.vv..>.>v.',
'v.v..>>v.v',
'....v..v.>']

filename = 'prob25input.txt'
lines = get_string_input(filename)

# get matrix of chars
M = [[ch for ch in l] for l in lines]

def stepMRight(M):
    ''' Step the pieces in matrix in M down if possible.
        Return the number of pieces that moved and the matrix M
    '''
    retval = []  # This is the new matrix M
    moved = 0
    n, m = len(M), len(M[0])
    for i in range(n):          # copy the matrix M
        retval.append(M[i][:])
        
    for i in range(n):
        for j in range(m):
            # if piece at row and col pointing right and there is room to move
            if M[i][j] == '>' and M[i][(j+1)%m] == '.':
                # move the piece right
                retval[i][j] = '.'
                retval[i][(j+1)%m] = '>'
                moved += 1
    return moved, retval

def stepMDown(M):
    ''' Step the pieces in matrix in M down if possible.
        Return the number of pieces that moved and the matrix M
    '''
    retval = []  # This is the new matrix M
    moved = 0
    n, m = len(M), len(M[0])
    for i in range(n):  # copy the matrix M
        retval.append(M[i][:])

    for i in range(n):
        for j in range(m):
            # if piece at row and col pointing down and there is room to move
            if M[i][j] == 'v' and M[(i+1)%n][j] == '.':
                # move the piece down
                retval[i][j] = '.'
                retval[(i+1)%n][j] = 'v'
                moved += 1
    return moved, retval

def stepM(M):
    ''' Step the matrix in M, first right then down.
        Return the number of pieces that moved and the matrix M
    '''
    moved_right, M = stepMRight(M)
    moved_left, M = stepMDown(M)
    return moved_right + moved_left, M


logging.info("Beginning sea cucumber movement.")
reallyBigNumber = 1000
for step in range(reallyBigNumber):
    moved, M = stepM(M)
    if moved == 0:
        break
        
logging.info("25a. No sea cucumbers moved after step: {}".format(step+1,))
logging.info("25b. Free Star!")

2023-11-08 06:10:02 INFO: Got 137 lines from prob25input.txt.
2023-11-08 06:10:02 INFO: Beginning sea cucumber movement.
2023-11-08 06:10:04 INFO: 25a. No sea cucumbers moved after step: 598
2023-11-08 06:10:04 INFO: 25b. Free Star!
